# Flight Search Agent Tutorial

This notebook demonstrates the Agent Catalog flight search agent using LangGraph with Couchbase vector store and Arize evaluation. Uses AI services with standard OpenAI wrappers and Capella (simple & fast).

The agent provides comprehensive flight search capabilities including:
- Flight lookup and search
- Flight booking management
- Airline review search
- Booking retrieval and management


In [1]:
import os
print(os.getcwd())

/content


In [2]:
# Download required resources for the flight search agent
!mkdir -p prompts
!wget -O prompts/flight_search_assistant.yaml https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/prompts/flight_search_assistant.yaml
!mkdir -p tools
!wget -O tools/lookup_flight_info.py https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/lookup_flight_info.py
!wget -O tools/retrieve_flight_bookings.py https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/retrieve_flight_bookings.py
!wget -O tools/save_flight_booking.py https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/save_flight_booking.py
!wget -O tools/search_airline_reviews.py https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/search_airline_reviews.py
!wget -O tools/schemas.py https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/schemas.py
!wget -O agentcatalog_index.json https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/agentcatalog_index.json
!wget -O .agentcignore https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/.agentcignore


--2025-10-23 15:57:59--  https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/prompts/flight_search_assistant.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9279 (9.1K) [text/plain]
Saving to: ‘prompts/flight_search_assistant.yaml’

prompts/flight_sear 100%[===================>]   9.06K  --.-KB/s    in 0.001s  

2025-10-23 15:57:59 (15.6 MB/s) - ‘prompts/flight_search_assistant.yaml’ saved [9279/9279]

--2025-10-23 15:58:00--  https://raw.githubusercontent.com/couchbase-examples/agent-catalog-quickstart/refs/heads/main/notebooks/flight_search_agent_langraph/tools/lookup_flight_info.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.19

In [3]:
%pip install -q \
    "pydantic>=2.0.0,<3.0.0" \
    "pydantic-settings>=2.10.1,<3.0.0" \
    "python-dotenv>=1.0.0,<2.0.0" \
    "pandas>=2.0.0,<3.0.0" \
    "nest-asyncio>=1.6.0,<2.0.0" \
    "uvicorn>=0.29.0,<0.30.0" \
    "kagglehub>=0.2.0,<1.0.0" \
    "langchain-couchbase>=0.4.0,<0.5.0" \
    "langchain-openai>=0.3.11,<0.4.0" \
    "langchain-nvidia-ai-endpoints>=0.3.13,<0.4.0" \
    "langgraph>=0.5.1,<0.6.0" \
    "arize>=7.51.0,<8.0.0" \
    "arize-phoenix>=11.37.0,<12.0.0" \
    "arize-phoenix-evals>=2.2.0,<3.0.0" \
    "openinference-instrumentation-langchain>=0.1.29,<0.2.0" \
    "openinference-instrumentation-openai>=0.1.18,<0.2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 k

In [4]:
%pip install -q https://github.com/couchbaselabs/agent-catalog/releases/download/v0.2.5a3/agentc_core-0.2.5a3-py3-none-any.whl
%pip install -q https://github.com/couchbaselabs/agent-catalog/releases/download/v0.2.5a3/agentc_cli-0.2.5a3-py3-none-any.whl
%pip install -q https://github.com/couchbaselabs/agent-catalog/releases/download/v0.2.5a3/agentc-0.2.5a3-py3-none-any.whl
%pip install -q https://github.com/couchbaselabs/agent-catalog/releases/download/v0.2.5a3/agentc_langchain-0.2.5a3-py3-none-any.whl # Explicitly install agentc_langchain a3
%pip install -q https://github.com/couchbaselabs/agent-catalog/releases/download/v0.2.5a3/agentc_langgraph-0.2.5a3-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [5]:
# Install the couchbase-infrastructure package
%pip install -q couchbase-infrastructure

## 🚀 Educational Infrastructure Setup

**This cell sets up your Couchbase Capella infrastructure step-by-step using the `couchbase-infrastructure` package.**

### What It Does:
1. **Prompts for Credentials** - Securely collects your API key (no .env file needed for Colab!)
2. **Creates Capella Project & Cluster** - Sets up your cloud database infrastructure
3. **Loads travel-sample Data** - Imports the sample dataset for the tutorial
4. **Deploys AI Models** - Provisions embedding (Mistral 7B) and LLM (Llama 3 8B) models
5. **Configures Network Access** - Sets up CIDR allowlists for connectivity
6. **Creates Database User** - Generates credentials with appropriate permissions
7. **Sets Environment Variables** - Configures all required variables for the tutorial

### You'll Be Prompted For:
- `MANAGEMENT_API_KEY` (required) - Get from [Capella Console](https://cloud.couchbase.com) → Settings → API Keys
- Optional: `ORGANIZATION_ID`, `PROJECT_NAME`, `CLUSTER_NAME` (defaults provided)

### Process Time:
⏳ This will take **10-15 minutes** for cluster and AI model deployment. You'll see step-by-step progress!

### After Running:
All subsequent cells will automatically use the provisioned infrastructure. No manual configuration needed!


In [6]:
import os
from getpass import getpass
from pathlib import Path

print("="*70)
print("🚀 Couchbase Capella Infrastructure Setup")
print("="*70)
print("\nThis educational setup shows you how to provision Capella infrastructure")
print("step-by-step using the couchbase-infrastructure package.\n")

# Import the infrastructure package
from couchbase_infrastructure import CapellaConfig, CapellaClient
from couchbase_infrastructure.resources import (
    create_project,
    create_developer_pro_cluster,
    add_allowed_cidr,
    load_sample_data,
    create_database_user,
    deploy_ai_model,
    create_ai_api_key,
)

# Step 1: Load from .env file if available, then collect any missing credentials
print("\n📋 Step 1: Collecting Credentials")
print("-"*70)

# Try to load .env file
env_file = Path('.env')
if env_file.exists():
    print("✅ Found .env file. Loading configuration...\n")
    from dotenv import load_dotenv
    load_dotenv('.env')
else:
    print("ℹ️  No .env file found. Will prompt for credentials.\n")

print("Get your credentials from: https://cloud.couchbase.com → Settings → API Keys\n")

# Required: MANAGEMENT_API_KEY
management_api_key = os.getenv('MANAGEMENT_API_KEY')
if management_api_key:
    print("✅ Using MANAGEMENT_API_KEY from environment")
else:
    management_api_key = getpass("Enter your MANAGEMENT_API_KEY (hidden): ")
    if not management_api_key:
        raise ValueError("MANAGEMENT_API_KEY is required!")

# Required: ORGANIZATION_ID
organization_id = os.getenv('ORGANIZATION_ID')
if organization_id:
    print(f"✅ Using ORGANIZATION_ID from environment: {organization_id}")
else:
    organization_id = input("Enter your ORGANIZATION_ID (required): ").strip()
    if not organization_id:
        raise ValueError("ORGANIZATION_ID is required! Find it in Capella Console under Settings.")

# Optional configuration (use env vars if available, otherwise prompt with defaults)
api_base_url = os.getenv('API_BASE_URL') or input("Enter API_BASE_URL (default: 'cloudapi.cloud.couchbase.com'): ").strip() or "cloudapi.cloud.couchbase.com"
project_name = os.getenv('PROJECT_NAME') or input("Enter PROJECT_NAME (default: 'agent-app'): ").strip() or "agent-app"
cluster_name = os.getenv('CLUSTER_NAME') or input("Enter CLUSTER_NAME (default: 'agent-app-cluster'): ").strip() or "agent-app-cluster"
db_username = os.getenv('DB_USERNAME') or input("Enter DB_USERNAME (default: 'agent_app_user'): ").strip() or "agent_app_user"
sample_bucket = os.getenv('SAMPLE_BUCKET') or input("Enter BUCKET_NAME (default: 'travel-sample'): ").strip() or "travel-sample"
embedding_model = os.getenv('EMBEDDING_MODEL_NAME') or input("Enter EMBEDDING_MODEL (default: 'nvidia/llama-3.2-nv-embedqa-1b-v2'): ").strip() or "nvidia/llama-3.2-nv-embedqa-1b-v2"
llm_model = os.getenv('LLM_MODEL_NAME') or input("Enter LLM_MODEL (default: 'meta/llama3-8b-instruct'): ").strip() or "meta/llama3-8b-instruct"

print("\n✅ Configuration collected successfully!\n")

# Step 2: Initialize configuration
print("\n🔧 Step 2: Initializing Configuration")
print("-"*70)
config = CapellaConfig(
    management_api_key=management_api_key,
    organization_id=organization_id,
    api_base_url=api_base_url,
    project_name=project_name,
    cluster_name=cluster_name,
    db_username=db_username,
    sample_bucket=sample_bucket,
    embedding_model_name=embedding_model,
    llm_model_name=llm_model,
)
print("✅ Configuration initialized\n")

# Step 3: Initialize client and get organization ID
print("\n🔌 Step 3: Initializing Client")
print("-"*70)
client = CapellaClient(config)
org_id = client.get_organization_id()
print(f"✅ Using Organization ID: {org_id}\n")

# Step 4: Test API connection
print("\n🔍 Step 4: Testing API Connection")
print("-"*70)
if not client.test_connection(org_id):
    raise ConnectionError("Failed to connect to Capella API")
print("✅ API connection successful\n")

# Step 5: Create Capella Project
print("\n📁 Step 5: Creating Capella Project")
print("-"*70)
project_id = create_project(client, org_id, config.project_name)
print(f"✅ Project ready: {config.project_name} (ID: {project_id})\n")

# Step 6: Create Developer Pro cluster with Analytics
print("\n☁️ Step 6: Creating Developer Pro Cluster with Analytics")
print("-"*70)
print("⏳ This will take 10-15 minutes for cluster deployment...\n")
cluster_id = create_developer_pro_cluster(
    client, org_id, project_id, config.cluster_name,
    cloud_provider=config.cluster_cloud_provider,
    region=config.cluster_region,
    cidr=config.cluster_cidr
)
# Wait for cluster to be ready
cluster_check_url = f"/v4/organizations/{org_id}/projects/{project_id}/clusters/{cluster_id}"
cluster_details = client.wait_for_resource(cluster_check_url, "Cluster", None)
cluster_conn_string = cluster_details.get("connectionString")
print(f"✅ Cluster ready: {config.cluster_name} (ID: {cluster_id})\n")

# Step 7: Configure network access
print("\n🌐 Step 7: Configuring Network Access")
print("-"*70)
add_allowed_cidr(client, org_id, project_id, cluster_id, config.allowed_cidr)
print("✅ Network access configured (0.0.0.0/0 allowed)\n")

# Step 8: Load travel-sample bucket
print("\n📦 Step 8: Loading travel-sample Bucket")
print("-"*70)
load_sample_data(client, org_id, project_id, cluster_id, config.sample_bucket)
print(f"✅ Sample data loaded: {config.sample_bucket}\n")

# Step 9: Create database user (password auto-generated)
print("\n👤 Step 9: Creating Database User")
print("-"*70)
db_password = create_database_user(
    client,
    org_id,
    project_id,
    cluster_id,
    config.db_username,
    config.sample_bucket,
    recreate_if_exists=True,  # Delete and recreate if exists to get fresh password
)
print(f"✅ Database user created: {config.db_username}\n")
if db_password and db_password != "existing_user_password_not_retrievable":
    print(f"   Auto-generated password: {db_password[:4]}...{db_password[-4:]}\n")

# Step 10: Deploy AI models
print("\n🤖 Step 10: Deploying AI Models")
print("-"*70)
print("⏳ Deploying embedding and LLM models (5-10 minutes)...\n")

# Deploy Embedding Model
print("   Deploying embedding model...")
embedding_model_id = deploy_ai_model(
    client,
    org_id,
    config.embedding_model_name,
    "agent-hub-embedding-model",
    "embedding",
    config,
)
embedding_check_url = f"/v4/organizations/{org_id}/aiServices/models/{embedding_model_id}"
embedding_details = client.wait_for_resource(embedding_check_url, "Embedding Model", None)

# Extract endpoint from nested 'model' object
model_info = embedding_details.get("model", {})
embedding_endpoint = model_info.get("connectionString", "")

print(f"✅ Embedding model deployed: {config.embedding_model_name}")
print(f"   Endpoint: {embedding_endpoint}\n")

# Deploy LLM Model
print("   Deploying LLM model...")
llm_model_id = deploy_ai_model(
    client,
    org_id,
    config.llm_model_name,
    "agent-hub-llm-model",
    "llm",
    config,
)
llm_check_url = f"/v4/organizations/{org_id}/aiServices/models/{llm_model_id}"
llm_details = client.wait_for_resource(llm_check_url, "LLM Model", None)

# Extract endpoint from nested 'model' object
model_info = llm_details.get("model", {})
llm_endpoint = model_info.get("connectionString", "")

print(f"✅ LLM model deployed: {config.llm_model_name}")
print(f"   Endpoint: {llm_endpoint}\n")

# Step 11: Create API Key for AI models
print("\n🔑 Step 11: Creating API Key for AI Models")
print("-"*70)
api_key = create_ai_api_key(client, org_id, config.ai_model_region)
print(f"✅ AI API key created\n")

# Step 12: Set environment variables
print("\n⚙️ Step 12: Setting Environment Variables")
print("-"*70)

# Set all environment variables for subsequent cells
# Ensure connection string has proper protocol
if not cluster_conn_string.startswith("couchbase://") and not cluster_conn_string.startswith("couchbases://"):
    cluster_conn_string = f"couchbases://{cluster_conn_string}"
    print(f"⚠️  Added protocol to connection string: {cluster_conn_string}")

os.environ["CB_CONN_STRING"] = cluster_conn_string + "?tls_verify=none"
os.environ["CB_USERNAME"] = config.db_username
os.environ["CB_PASSWORD"] = db_password
os.environ["CB_BUCKET"] = config.sample_bucket
os.environ["CAPELLA_API_ENDPOINT"] = embedding_endpoint  # Use as base endpoint
os.environ["CAPELLA_API_EMBEDDING_ENDPOINT"] = embedding_endpoint
os.environ["CAPELLA_API_LLM_ENDPOINT"] = llm_endpoint
os.environ["CAPELLA_API_EMBEDDINGS_KEY"] = api_key
os.environ["CAPELLA_API_LLM_KEY"] = api_key
os.environ["CAPELLA_API_EMBEDDING_MODEL"] = config.embedding_model_name
os.environ["CAPELLA_API_LLM_MODEL"] = config.llm_model_name

print("✅ Environment variables configured:\n")
print(f"   CB_CONN_STRING: {cluster_conn_string}")
print(f"   CB_USERNAME: {config.db_username}")
print(f"   CB_BUCKET: {config.sample_bucket}")
print(f"   CAPELLA_API_EMBEDDING_ENDPOINT: {embedding_endpoint}")
print(f"   CAPELLA_API_LLM_ENDPOINT: {llm_endpoint}")
print(f"   CAPELLA_API_EMBEDDING_MODEL: {config.embedding_model_name}")
print(f"   CAPELLA_API_LLM_MODEL: {config.llm_model_name}")

print("\n" + "="*70)
print("✅ Infrastructure Setup Complete!")
print("="*70)
print("\nYou can now run the flight search agent cells below.\n")


🚀 Couchbase Capella Infrastructure Setup

This educational setup shows you how to provision Capella infrastructure
step-by-step using the couchbase-infrastructure package.


📋 Step 1: Collecting Credentials
----------------------------------------------------------------------
✅ Found .env file. Loading configuration...

Get your credentials from: https://cloud.couchbase.com → Settings → API Keys

✅ Using MANAGEMENT_API_KEY from environment
✅ Using ORGANIZATION_ID from environment: 23086345-371f-4650-8dc4-c61733dd27a0
Enter PROJECT_NAME (default: 'agent-app'): 
Enter CLUSTER_NAME (default: 'agent-app-cluster'): 
Enter DB_USERNAME (default: 'agent_app_user'): 
Enter BUCKET_NAME (default: 'travel-sample'): 
Enter EMBEDDING_MODEL (default: 'nvidia/llama-3.2-nv-embedqa-1b-v2'): 
Enter LLM_MODEL (default: 'meta/llama3-8b-instruct'): 

✅ Configuration collected successfully!


🔧 Step 2: Initializing Configuration
----------------------------------------------------------------------
✅ Config

🔍 Testing API connection...

Current IP: 35.188.74.140

API Base URL: https://cloudapi.sbx-29.sandbox.nonprod-project-avengers.com

Organization ID: 23086345-371f-4650-8dc4-c61733dd27a0

...

Creating API key for models in region us-east-1...

✅ API key created successfully.

Key ID: e21f3a18-cabf-56c9-8576-3c21a01e44df

Token: cbsk-v1-MZgnu4Gnr2jo...

✅ AI API key created


⚙️ Step 12: Setting Environment Variables
----------------------------------------------------------------------
⚠️  Added protocol to connection string: couchbases://cb.maggf6p2qyrvsjib.sandbox.nonprod-project-avengers.com
✅ Environment variables configured:

   CB_CONN_STRING: couchbases://cb.maggf6p2qyrvsjib.sandbox.nonprod-project-avengers.com
   CB_USERNAME: agent_app_user
   CB_BUCKET: travel-sample
   CAPELLA_API_EMBEDDING_ENDPOINT: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com
   CAPELLA_API_LLM_ENDPOINT: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com
   CAPELLA_API_EMBEDDING_MODEL: nvidia/llama-3.2-nv-embedqa-1b-v2
   CAPELLA_API_LLM_MODEL: meta/llama3-8b-instruct

✅ Infrastructure Setup Complete!

You can now run the flight search agent cells below.



In [7]:
# Set Agent Catalog environment variables (required for agentc commands)
# These use the same Couchbase connection created above
import os

# Strip the ?tls_verify=none from the connection string for agentc
conn_string = os.environ["CB_CONN_STRING"]
if conn_string.endswith("?tls_verify=none"):
    conn_string_clean = conn_string[:-len("?tls_verify=none")]
else:
    conn_string_clean = conn_string

# Ensure connection string has proper protocol (agentc requires couchbase:// or couchbases://)
if not conn_string_clean.startswith("couchbase://") and not conn_string_clean.startswith("couchbases://"):
    # Add couchbases:// protocol for secure connections
    conn_string_clean = f"couchbases://{conn_string_clean}"
    print(f"⚠️  Added protocol to connection string: {conn_string_clean}")

os.environ["AGENT_CATALOG_CONN_STRING"] = conn_string_clean

os.environ["AGENT_CATALOG_USERNAME"] = os.environ["CB_USERNAME"]
os.environ["AGENT_CATALOG_PASSWORD"] = os.environ["CB_PASSWORD"]
os.environ["AGENT_CATALOG_BUCKET"] = os.environ["CB_BUCKET"]

print("✅ Agent Catalog environment variables set:")
print(f"   AGENT_CATALOG_CONN_STRING: {os.environ['AGENT_CATALOG_CONN_STRING']}")
print(f"   AGENT_CATALOG_USERNAME: {os.environ['AGENT_CATALOG_USERNAME']}")
print(f"   AGENT_CATALOG_BUCKET: {os.environ['AGENT_CATALOG_BUCKET']}")

# Handle root certificate (required for secure connections)
print("\n" + "="*70)
print("📜 Root Certificate Setup")
print("="*70)
print("\n⚠️  IMPORTANT: You need to download the root certificate from Capella UI")
print("\nSteps:")
print("1. Go to Capella Console: https://cloud.couchbase.com")
print("2. Navigate to your cluster → Connect tab")
print("3. Download the 'Root Certificate' file")
print("4. Upload it using the file upload below\n")

# Try to use Google Colab's file upload, fallback to manual input
try:
    from google.colab import files
    print("📤 Please upload your root certificate file:")
    uploaded = files.upload()

    if uploaded:
        cert_filename = list(uploaded.keys())[0]
        # Validate it's actually a certificate file
        if cert_filename.endswith(('.pem', '.crt', '.cer', '.txt')):
            os.environ["AGENT_CATALOG_CONN_ROOT_CERTIFICATE"] = cert_filename
            print(f"\n✅ Root certificate uploaded: {cert_filename}")
            print(f"   AGENT_CATALOG_CONN_ROOT_CERTIFICATE: {cert_filename}")
        else:
            print(f"\n⚠️  Uploaded file '{cert_filename}' doesn't appear to be a certificate (.pem, .crt, .cer, .txt)")
            print("   Skipping certificate setup. You can configure it later if needed.")
            os.environ["AGENT_CATALOG_CONN_ROOT_CERTIFICATE"] = ""
    else:
        print("\n⚠️  No file uploaded. You can set it manually later if needed.")
        os.environ["AGENT_CATALOG_CONN_ROOT_CERTIFICATE"] = ""
except ImportError:
    # Not in Colab - ask user to place file and provide filename
    print("📝 Not running in Google Colab.")
    print("   Please place the root certificate file in the current directory.\n")
    cert_filename = input("Enter the certificate filename (or press Enter to skip): ").strip()

    if cert_filename:
        os.environ["AGENT_CATALOG_CONN_ROOT_CERTIFICATE"] = cert_filename
        print(f"\n✅ Root certificate set: {cert_filename}")
    else:
        print("\n⚠️  Root certificate not set. You can add it manually later if needed.")
        os.environ["AGENT_CATALOG_CONN_ROOT_CERTIFICATE"] = ""

print("\n" + "="*70)
print("✅ Agent Catalog Configuration Complete")
print("="*70)

# Write environment variables to .env file for agentc commands
# agentc CLI will load from .env file automatically
import os.path
with open('.env', 'w') as f:
    # CB variables (needed for database operations - prevents wiping by dotenv.load_dotenv)
    f.write(f"CB_CONN_STRING={os.environ['CB_CONN_STRING']}\n")
    f.write(f"CB_USERNAME={os.environ['CB_USERNAME']}\n")
    f.write(f"CB_PASSWORD={os.environ['CB_PASSWORD']}\n")
    f.write(f"CB_BUCKET={os.environ['CB_BUCKET']}\n")

    # CAPELLA_API variables (needed for AI services - prevents wiping by dotenv.load_dotenv)
    f.write(f"CAPELLA_API_ENDPOINT={os.environ.get('CAPELLA_API_ENDPOINT', '')}\n")
    f.write(f"CAPELLA_API_EMBEDDING_ENDPOINT={os.environ.get('CAPELLA_API_EMBEDDING_ENDPOINT', '')}\n")
    f.write(f"CAPELLA_API_LLM_ENDPOINT={os.environ.get('CAPELLA_API_LLM_ENDPOINT', '')}\n")
    f.write(f"CAPELLA_API_EMBEDDINGS_KEY={os.environ.get('CAPELLA_API_EMBEDDINGS_KEY', '')}\n")
    f.write(f"CAPELLA_API_LLM_KEY={os.environ.get('CAPELLA_API_LLM_KEY', '')}\n")
    f.write(f"CAPELLA_API_EMBEDDING_MODEL={os.environ.get('CAPELLA_API_EMBEDDING_MODEL', '')}\n")
    f.write(f"CAPELLA_API_LLM_MODEL={os.environ.get('CAPELLA_API_LLM_MODEL', '')}\n")

    # AGENT_CATALOG variables (for agentc CLI)
    f.write(f"AGENT_CATALOG_CONN_STRING={os.environ['AGENT_CATALOG_CONN_STRING']}\n")
    f.write(f"AGENT_CATALOG_USERNAME={os.environ['AGENT_CATALOG_USERNAME']}\n")
    f.write(f"AGENT_CATALOG_PASSWORD={os.environ['AGENT_CATALOG_PASSWORD']}\n")
    f.write(f"AGENT_CATALOG_BUCKET={os.environ['AGENT_CATALOG_BUCKET']}\n")

    # Only write certificate if it exists and is a valid file
    cert = os.environ.get('AGENT_CATALOG_CONN_ROOT_CERTIFICATE', '').strip()
    if cert and os.path.isfile(cert):
        f.write(f"AGENT_CATALOG_CONN_ROOT_CERTIFICATE={cert}\n")
    elif cert:
        print(f"⚠️  Warning: Certificate file '{cert}' not found, skipping from .env")

print("\n✅ Environment variables written to .env file for agentc commands")

# Verify .env file was created correctly
print("\n🔍 Verifying .env file contents:")
!pwd
!ls -la .env
print("\nFirst 5 lines of .env (passwords masked):")
with open('.env', 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            if 'PASSWORD' in line:
                print(f"  {line.split('=')[0]}=***")
            else:
                print(f"  {line.strip()}")

✅ Agent Catalog environment variables set:
   AGENT_CATALOG_CONN_STRING: couchbases://cb.maggf6p2qyrvsjib.sandbox.nonprod-project-avengers.com
   AGENT_CATALOG_USERNAME: agent_app_user
   AGENT_CATALOG_BUCKET: travel-sample

📜 Root Certificate Setup

⚠️  IMPORTANT: You need to download the root certificate from Capella UI

Steps:
1. Go to Capella Console: https://cloud.couchbase.com
2. Navigate to your cluster → Connect tab
3. Download the 'Root Certificate' file
4. Upload it using the file upload below

📤 Please upload your root certificate file:


Saving agent-app-cluster-root-certificate.txt to agent-app-cluster-root-certificate.txt

✅ Root certificate uploaded: agent-app-cluster-root-certificate.txt
   AGENT_CATALOG_CONN_ROOT_CERTIFICATE: agent-app-cluster-root-certificate.txt

✅ Agent Catalog Configuration Complete

✅ Environment variables written to .env file for agentc commands

🔍 Verifying .env file contents:
/content
-rw-r--r-- 1 root root 1022 Oct 23 16:01 .env

First 5 lines of .env (passwords masked):
  CB_CONN_STRING=couchbases://cb.maggf6p2qyrvsjib.sandbox.nonprod-project-avengers.com?tls_verify=none
  CB_USERNAME=agent_app_user
  CB_PASSWORD=***
  CB_BUCKET=travel-sample
  CAPELLA_API_ENDPOINT=https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com


### Optional: Configure OpenAI and Arize (Observability)

Provide optional API keys for:
- **OpenAI**: Fallback LLM/embeddings if Capella AI is unavailable
- **Arize Phoenix**: Observability and evaluation platform

In [8]:
import os
import getpass

print("="*70)
print("🔧 Optional API Keys Configuration")
print("="*70)

# OpenAI Configuration (optional - for fallback)
print("\n📝 OpenAI API (Optional - for fallback LLM/embeddings)")
print("-"*70)
print("Press Enter to skip, or provide your OpenAI API key:")
try:
    openai_api_key = getpass.getpass("OpenAI API Key: ").strip()
except:
    # Fallback for environments where getpass doesn't work
    openai_api_key = ""

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    os.environ["OPENAI_MODEL"] = "gpt-4o"  # Default model
    print("✅ OpenAI API key configured")
    print(f"   Model: gpt-4o")
else:
    print("⏭️  Skipped OpenAI configuration (will use Capella AI only)")
    os.environ["OPENAI_API_KEY"] = ""
    os.environ["OPENAI_MODEL"] = "gpt-4o"

# Arize Phoenix Configuration (optional - for observability)
print("\n📊 Arize Phoenix (Optional - for observability and evaluation)")
print("-"*70)
print("Press Enter to skip, or provide your Arize credentials:")
try:
    arize_space_id = getpass.getpass("Arize Space ID: ").strip()
    arize_api_key = getpass.getpass("Arize API Key: ").strip() if arize_space_id else ""
except:
    # Fallback for environments where getpass doesn't work
    arize_space_id = ""
    arize_api_key = ""

if arize_space_id and arize_api_key:
    os.environ["ARIZE_SPACE_ID"] = arize_space_id
    os.environ["ARIZE_API_KEY"] = arize_api_key
    print("✅ Arize Phoenix configured")
else:
    print("⏭️  Skipped Arize configuration (observability disabled)")
    os.environ["ARIZE_SPACE_ID"] = ""
    os.environ["ARIZE_API_KEY"] = ""

# Append optional variables to .env file
with open('.env', 'a') as f:
    f.write("\n# Optional: OpenAI Configuration (fallback LLM/embeddings)\n")
    f.write(f"OPENAI_API_KEY={os.environ['OPENAI_API_KEY']}\n")
    f.write(f"OPENAI_MODEL={os.environ['OPENAI_MODEL']}\n")

    f.write("\n# Optional: Arize Phoenix (observability and evaluation)\n")
    f.write(f"ARIZE_SPACE_ID={os.environ['ARIZE_SPACE_ID']}\n")
    f.write(f"ARIZE_API_KEY={os.environ['ARIZE_API_KEY']}\n")

print("\n" + "="*70)
print("✅ Optional Configuration Complete")
print("="*70)


🔧 Optional API Keys Configuration

📝 OpenAI API (Optional - for fallback LLM/embeddings)
----------------------------------------------------------------------
Press Enter to skip, or provide your OpenAI API key:
OpenAI API Key: ··········
✅ OpenAI API key configured
   Model: gpt-4o

📊 Arize Phoenix (Optional - for observability and evaluation)
----------------------------------------------------------------------
Press Enter to skip, or provide your Arize credentials:
Arize Space ID: ··········
Arize API Key: ··········
✅ Arize Phoenix configured

✅ Optional Configuration Complete


In [9]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [10]:
!git add .
!git config --global user.email "your.email@example.com"
!git config --global user.name "Your Name"
!git commit -m "initial commit"


[master (root-commit) 048941a] initial commit
 31 files changed, 52109 insertions(+)
 create mode 100644 .agentcignore
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.10.21/16.51.10.047293.log
 create mode 100644 .config/logs/2025.10.21/16.51.30.790200.log
 create mode 100644 .config/logs/2025.10.21/16.51.39.965567.log
 create mode 100644 .config/logs/2025.10.21/16.51.41.225539.log
 create mode 100644 .config/logs/2025.10.21/16.51.50.106792.log
 create mode 100644 .config/logs/2025.10.21/16.51.50.856899.log
 create mode 100644 .env


In [11]:
!agentc init


2025-10-23 16:02:20.568034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761235340.583772    1454 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761235340.588192    1454 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761235340.600424    1454 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761235340.600480    1454 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761235340.600485    1454 computation_placer.cc:177] computation placer alr

In [12]:
!agentc index .



TOOL

Crawling .:
.last_survey_prompt.yaml:   0% 0/8 [00:00<?, ?it/s]Encountered .yaml file with unknown record_kind field. Not indexing /content/.config/.last_survey_prompt.yaml.
.last_opt_in_prompt.yaml:   0% 0/8 [00:00<?, ?it/s]Encountered .yaml file with unknown record_kind field. Not indexing /content/.config/.last_opt_in_prompt.yaml.
flight_search_assistant.yaml: 100% 8/8 [00:04<00:00,  1.87it/s]

Generating embeddings:
save_flight_booking:   0% 0/4 [00:00<?, ?it/s]2025-10-23 16:02:59.302536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761235379.316517    1714 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761235379.320507    1714 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBL

In [13]:
!agentc publish



TOOL

Using the catalog identifier: 048941ad6fb885d9d3892c84f56b61878d05e633

Uploading the tool catalog items to Couchbase.
search_airline_reviews: 100% 4/4 [00:00<00:00, 24.95it/s]
Tool catalog items successfully uploaded to Couchbase!


PROMPT

Using the catalog identifier: 048941ad6fb885d9d3892c84f56b61878d05e633

Uploading the prompt catalog items to Couchbase.
flight_search_assistant: 100% 1/1 [00:00<00:00, 15.98it/s]
Prompt catalog items successfully uploaded to Couchbase!



## Setup and Imports

Import all necessary modules for the flight search agent using the latest code structure.


In [14]:
import json
import logging
import os
import sys
import time
from datetime import timedelta

import agentc
import agentc_langgraph.agent
import agentc_langgraph.graph
import dotenv
import langchain_core.messages
import langchain_core.runnables
import langchain_openai.chat_models
import langgraph.graph
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import KeyspaceNotFoundException
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from pydantic import SecretStr

# Setup logging for Colab (must use stdout instead of stderr)
root_logger = logging.getLogger()
if not root_logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)
logger = logging.getLogger(__name__)

# Suppress verbose logging from external libraries
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("agentc_core").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)


True

## Project Root Discovery and Shared Imports

Essential project root discovery and shared module imports exactly as in working main.py.


In [15]:
# INLINE IMPLEMENTATION - Universal AI Services and Couchbase Client
import base64
import time
from typing import Tuple, Any, Optional, List
from datetime import timedelta
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import KeyspaceNotFoundException
from couchbase.management.buckets import BucketType, CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions

# We'll add the function implementations in the next cell

## Universal AI Services Setup

5-case priority AI service setup implementation inline.


In [16]:
def setup_ai_services(
    framework: str = "langchain",
    temperature: float = 0.0,
    callbacks: Optional[List] = None,
    application_span: Optional[Any] = None
) -> Tuple[Any, Any]:
    """Priority 1 AI service setup - Capella with direct API keys + OpenAI wrappers only."""
    embeddings = None
    llm = None

    logger.info(f"🔧 Setting up Priority 1 AI services for {framework} framework...")

    # PRIORITY 1: LATEST CAPELLA (OpenAI wrappers with direct API keys)
    if (
        not embeddings
        and os.getenv("CAPELLA_API_ENDPOINT")
        and os.getenv("CAPELLA_API_EMBEDDINGS_KEY")
    ):
        try:
            if framework == "llamaindex":
                from llama_index.embeddings.openai import OpenAIEmbedding
                embeddings = OpenAIEmbedding(
                    api_key=os.getenv("CAPELLA_API_EMBEDDINGS_KEY"),
                    api_base=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                    model_name=os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
                    embed_batch_size=30,
                )
            else:  # langchain, langgraph
                from langchain_openai import OpenAIEmbeddings
                embeddings = OpenAIEmbeddings(
                    model=os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
                    api_key=os.getenv("CAPELLA_API_EMBEDDINGS_KEY"),
                    base_url=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                    check_embedding_ctx_length=False,  # Fix for asymmetric models
                )
            logger.info("✅ Using latest Capella AI embeddings (direct API key + OpenAI wrapper)")
        except Exception as e:
            logger.warning(f"⚠️ Latest Capella AI embeddings failed: {e}")

    if (
        not llm
        and os.getenv("CAPELLA_API_ENDPOINT")
        and os.getenv("CAPELLA_API_LLM_KEY")
    ):
        try:
            if framework == "llamaindex":
                from llama_index.llms.openai_like import OpenAILike
                llm = OpenAILike(
                    model=os.getenv("CAPELLA_API_LLM_MODEL"),
                    api_base=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                    api_key=os.getenv("CAPELLA_API_LLM_KEY"),
                    is_chat_model=True,
                    temperature=temperature,
                )
            else:  # langchain, langgraph
                from langchain_openai import ChatOpenAI

                chat_kwargs = {
                    "api_key": os.getenv("CAPELLA_API_LLM_KEY"),
                    "base_url": f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                    "model": os.getenv("CAPELLA_API_LLM_MODEL"),
                    "temperature": temperature,
                }
                if callbacks:
                    chat_kwargs["callbacks"] = callbacks

                llm = ChatOpenAI(**chat_kwargs)

            # Test the LLM works
            if framework == "llamaindex":
                llm.complete("Hello")
            else:
                llm.invoke("Hello")

            logger.info("✅ Using latest Capella AI LLM (direct API key + OpenAI wrapper)")
        except Exception as e:
            logger.warning(f"⚠️ Latest Capella AI LLM failed: {e}")
            llm = None

    # VALIDATION
    if not embeddings:
        raise ValueError("❌ No embeddings service could be initialized")
    if not llm:
        raise ValueError("❌ No LLM service could be initialized")

    logger.info(f"✅ AI services setup completed for {framework}")
    return embeddings, llm

## Helper Functions

Environment setup and connectivity test functions.


In [17]:
def setup_environment():
    """Setup default environment variables for agent operations."""
    defaults = {
        "CB_BUCKET": "travel-sample",
        "CB_SCOPE": "agentc_data",
        "CB_COLLECTION": "airline_reviews",
        "CB_INDEX": "airline_reviews_index",
        "CAPELLA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "CAPELLA_API_LLM_MODEL": "meta/llama-3-8b-instruct",
        "CAPELLA_API_EMBEDDING_MAX_TOKENS": "512",
        "NVIDIA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "NVIDIA_API_LLM_MODEL": "meta/llama-3.1-70b-instruct",
    }

    for key, value in defaults.items():
        if not os.getenv(key):
            os.environ[key] = value

    logger.info("✅ Environment variables configured")


def test_capella_connectivity(api_key: str = None, endpoint: str = None) -> bool:
    """Test connectivity to Capella AI services."""
    try:
        import httpx

        test_key = api_key or os.getenv("CAPELLA_API_EMBEDDINGS_KEY") or os.getenv("CAPELLA_API_LLM_KEY")
        test_endpoint = endpoint or os.getenv("CAPELLA_API_ENDPOINT")

        if not test_key or not test_endpoint:
            return False

        headers = {"Authorization": f"Bearer {test_key}"}

        with httpx.Client(timeout=10.0) as client:
            response = client.get(f"{test_endpoint.rstrip('/')}/v1/models", headers=headers)
            return response.status_code < 500

    except Exception as e:
        logger.warning(f"⚠️ Capella connectivity test failed: {e}")
        return False

## Universal Couchbase Client

Complete Couchbase client implementation with database operations.


In [18]:
class CouchbaseClient:
    """Universal Couchbase client for all database operations across agent frameworks."""

    def __init__(
        self,
        conn_string: str,
        username: str,
        password: str,
        bucket_name: str,
        wan_profile: bool = True,
        timeout_seconds: int = 20,
    ):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.wan_profile = wan_profile
        self.timeout_seconds = timeout_seconds
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)

            if self.wan_profile:
                options.apply_profile("wan_development")

            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=self.timeout_seconds))
            logger.info("✅ Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"❌ Failed to connect to Couchbase: {e!s}")

    def setup_bucket(self, create_if_missing: bool = True):
        """Setup bucket - connect to existing or create if missing."""
        try:
            if not self.cluster:
                self.connect()

            try:
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"✅ Connected to existing bucket '{self.bucket_name}'")
                return self.bucket
            except Exception as e:
                logger.info(f"⚠️ Bucket '{self.bucket_name}' not accessible: {e}")

            if create_if_missing:
                logger.info(f"🔧 Creating bucket '{self.bucket_name}'...")
                bucket_settings = CreateBucketSettings(
                    name=self.bucket_name,
                    bucket_type=BucketType.COUCHBASE,
                    ram_quota_mb=1024,
                    flush_enabled=True,
                    num_replicas=0,
                )
                self.cluster.buckets().create_bucket(bucket_settings)
                time.sleep(5)
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"✅ Bucket '{self.bucket_name}' created successfully")
                return self.bucket
            else:
                raise RuntimeError(f"❌ Bucket '{self.bucket_name}' not found")

        except Exception as e:
            raise RuntimeError(f"❌ Error setting up bucket: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str, clear_existing_data: bool = True, create_primary_index: bool = True):
        """Setup collection with comprehensive options."""
        try:
            if not self.bucket:
                self.setup_bucket()

            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"🔧 Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"✅ Scope '{scope_name}' created successfully")

            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists:
                if clear_existing_data:
                    logger.info(f"🗑️ Collection '{collection_name}' exists, clearing data...")
                    self.clear_collection_data(scope_name, collection_name)
                else:
                    logger.info(f"ℹ️ Collection '{collection_name}' exists, keeping existing data")
            else:
                logger.info(f"🔧 Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"✅ Collection '{collection_name}' created successfully")

            time.sleep(3)

            if create_primary_index:
                try:
                    self.cluster.query(
                        f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                    ).execute()
                    logger.info("✅ Primary index created successfully")
                except Exception as e:
                    logger.warning(f"⚠️ Error creating primary index: {e}")

            collection_key = f"{scope_name}.{collection_name}"
            collection = self.bucket.scope(scope_name).collection(collection_name)
            self._collections[collection_key] = collection

            logger.info(f"✅ Collection setup complete: {scope_name}.{collection_name}")
            return collection

        except Exception as e:
            raise RuntimeError(f"❌ Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str, verify_cleared: bool = True):
        """Clear all data from a collection."""
        try:
            logger.info(f"🗑️ Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")

            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)
            list(result)

            time.sleep(2)

            if verify_cleared:
                count_query = f"SELECT COUNT(*) as count FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                count_result = self.cluster.query(count_query)
                count_row = list(count_result)[0]
                remaining_count = count_row["count"]

                if remaining_count == 0:
                    logger.info(f"✅ Collection cleared successfully")
                else:
                    logger.warning(f"⚠️ Collection clear incomplete, {remaining_count} documents remaining")

        except KeyspaceNotFoundException:
            logger.info(f"ℹ️ Collection doesn't exist, nothing to clear")
        except Exception as e:
            logger.warning(f"⚠️ Error clearing collection data: {e}")

    def clear_scope(self, scope_name: str):
        """Clear all collections in the specified scope."""
        try:
            if not self.bucket:
                self.setup_bucket()

            logger.info(f"🗑️ Clearing scope: {self.bucket_name}.{scope_name}")
            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()

            target_scope = None
            for scope in scopes:
                if scope.name == scope_name:
                    target_scope = scope
                    break

            if not target_scope:
                logger.info(f"ℹ️ Scope '{self.bucket_name}.{scope_name}' does not exist")
                return

            for collection in target_scope.collections:
                try:
                    self.clear_collection_data(scope_name, collection.name, verify_cleared=False)
                    logger.info(f"✅ Cleared collection: {collection.name}")
                except Exception as e:
                    logger.warning(f"⚠️ Could not clear collection {collection.name}: {e}")

            logger.info(f"✅ Completed clearing scope: {self.bucket_name}.{scope_name}")

        except Exception as e:
            logger.warning(f"❌ Could not clear scope: {e}")

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index for the specified scope."""
        try:
            if not self.bucket:
                raise RuntimeError("❌ Bucket not initialized. Call setup_bucket first.")

            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"🔧 Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"✅ Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"ℹ️ Vector search index '{index_name}' already exists")
        except Exception as e:
            raise RuntimeError(f"❌ Error setting up vector search index: {e!s}")

    def load_index_definition(self, index_file_path: str = "agentcatalog_index.json"):
        """Load vector search index definition from JSON file."""
        try:
            with open(index_file_path) as file:
                index_definition = json.load(file)
            logger.info(f"✅ Loaded index definition from {index_file_path}")
            return index_definition
        except FileNotFoundError:
            logger.warning(f"⚠️ {index_file_path} not found")
            return None
        except Exception as e:
            logger.warning(f"⚠️ Error loading index definition: {e!s}")
            return None

    def setup_vector_store_langchain(self, scope_name: str, collection_name: str, index_name: str, embeddings, data_loader_func=None, **loader_kwargs):
        """Setup LangChain CouchbaseSearchVectorStore with optional data loading."""
        try:
            from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore

            if data_loader_func:
                logger.info("🔄 Loading data into vector store...")
                data_loader_func(
                    cluster=self.cluster,
                    bucket_name=self.bucket_name,
                    scope_name=scope_name,
                    collection_name=collection_name,
                    embeddings=embeddings,
                    index_name=index_name,
                    **loader_kwargs,
                )
                logger.info("✅ Data loaded into vector store successfully")

            vector_store = CouchbaseSearchVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embedding=embeddings,
                index_name=index_name,
            )

            logger.info(f"✅ LangChain vector store setup complete")
            return vector_store

        except Exception as e:
            raise RuntimeError(f"❌ Error setting up LangChain vector store: {e!s}")


def create_couchbase_client(
    conn_string: str = None,
    username: str = None,
    password: str = None,
    bucket_name: str = None,
    wan_profile: bool = True,
    timeout_seconds: int = 20,
) -> CouchbaseClient:
    """Factory function to create CouchbaseClient with environment variable defaults."""
    return CouchbaseClient(
        conn_string=conn_string or os.getenv("CB_CONN_STRING", "couchbase://localhost"),
        username=username or os.getenv("CB_USERNAME", "Administrator"),
        password=password or os.getenv("CB_PASSWORD", "password"),
        bucket_name=bucket_name or os.getenv("CB_BUCKET", "travel-sample"),
        wan_profile=wan_profile,
        timeout_seconds=timeout_seconds,
    )

## Shared Flight Search Queries

Exact code from queries.py - flight search queries and reference answers.


In [19]:
# Flight search queries (for evaluation and testing)
FLIGHT_SEARCH_QUERIES = [
    "Find flights from JFK to LAX",
    "Book a flight from LAX to JFK for tomorrow, 2 passengers, business class",
    "Book an economy flight from JFK to MIA for next week, 1 passenger",
    "Show me my current flight bookings",
    "What do passengers say about SpiceJet's service quality?",
]

# Comprehensive reference answers based on actual system responses
FLIGHT_REFERENCE_ANSWERS = [
    # Query 1: Flight search JFK to LAX
    """Available flights from JFK to LAX:

1. AS flight from JFK to LAX using 321 762
2. B6 flight from JFK to LAX using 320
3. DL flight from JFK to LAX using 76W 752
4. QF flight from JFK to LAX using 744
5. AA flight from JFK to LAX using 32B 762
6. UA flight from JFK to LAX using 757
7. US flight from JFK to LAX using 32B 762
8. VX flight from JFK to LAX using 320""",

    # Query 2: Flight booking LAX to JFK for tomorrow, 2 passengers, business class
    # Note: Departure date and Booking ID are dynamically generated based on current date
    """Flight Booking Confirmed!

Booking ID: [Dynamically Generated]
Route: LAX → JFK
Departure Date: [Tomorrow's Date - Dynamically Calculated]
Passengers: 2
Class: business
Total Price: $1500.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID

Thank you for choosing our airline!""",

    # Query 3: Flight booking JFK to MIA for next week
    # Note: Departure date and Booking ID are dynamically generated based on current date
    """Flight Booking Confirmed!

Booking ID: [Dynamically Generated]
Route: JFK → MIA
Departure Date: [Next Week's Date - Dynamically Calculated]
Passengers: 1
Class: economy
Total Price: $250.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID

Thank you for choosing our airline!""",

    # Query 4: Show current flight bookings
    # Note: Booking IDs and dates are dynamically generated
    """Your Current Bookings (2 found):

Booking 1:
  Booking ID: [Dynamically Generated]
  Route: LAX → JFK
  Date: [Tomorrow's Date - Dynamically Calculated]
  Passengers: 2
  Class: business
  Total: $1500.00
  Status: Confirmed
  Booked: [Current Date]

Booking 2:
  Booking ID: [Dynamically Generated]
  Route: JFK → MIA
  Date: [Next Week's Date - Dynamically Calculated]
  Passengers: 1
  Class: economy
  Total: $250.00
  Status: Confirmed
  Booked: [Current Date]""",

    # Query 5: SpiceJet service quality reviews
    # Note: Vector search results are non-deterministic and may return different reviews each time
    # from the database of 2210+ airline reviews. All results are valid SpiceJet reviews.
    """Found 5 relevant airline reviews for 'SpiceJet service quality':

[Reviews include customer feedback about SpiceJet's service quality, covering aspects such as:
- Crew service and hospitality
- Flight delays and punctuality
- In-flight amenities and comfort
- Ground staff service
- Overall travel experience
- Ratings typically range from 2/10 to 10/10
- Mix of verified and unverified reviews
- Reviews from various dates and routes
- Both positive recommendations and criticisms]

Note: Specific reviews vary due to vector similarity search across 2210+ airline reviews.""",
]

# Create dictionary for backward compatibility
QUERY_REFERENCE_ANSWERS = {
    query: answer for query, answer in zip(FLIGHT_SEARCH_QUERIES, FLIGHT_REFERENCE_ANSWERS)
}

def get_test_queries():
    """Return test queries for evaluation."""
    return FLIGHT_SEARCH_QUERIES

def get_evaluation_queries():
    """Get queries for evaluation"""
    return FLIGHT_SEARCH_QUERIES

def get_all_queries():
    """Get all available queries"""
    return FLIGHT_SEARCH_QUERIES

def get_simple_queries():
    """Get simple queries for basic testing"""
    return FLIGHT_SEARCH_QUERIES

def get_flight_policy_queries():
    """Return flight policy queries (for backward compatibility)."""
    return FLIGHT_SEARCH_QUERIES

def get_reference_answer(query: str) -> str:
    """Get the correct reference answer for a given query"""
    return QUERY_REFERENCE_ANSWERS.get(query, f"No reference answer available for: {query}")

def get_all_query_references():
    """Get all query-reference pairs"""
    return QUERY_REFERENCE_ANSWERS

## Airline Reviews Data Module

Exact code from airline_reviews_data.py - data loading and processing.


In [20]:
import pandas as pd
from tqdm import tqdm
from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore

# Import kagglehub only when needed
try:
    import kagglehub
except ImportError:
    kagglehub = None

class AirlineReviewsDataManager:
    """Manages airline reviews data loading, processing, and embedding."""

    def __init__(self):
        self._raw_data_cache = None
        self._processed_texts_cache = None

    def load_raw_data(self):
        """Load raw airline reviews data from Kaggle dataset (with caching)."""
        if self._raw_data_cache is not None:
            return self._raw_data_cache

        try:
            if kagglehub is None:
                raise ImportError("kagglehub is not available")

            logger.info("Downloading Indian Airlines Customer Reviews dataset from Kaggle...")
            path = kagglehub.dataset_download("jagathratchakan/indian-airlines-customer-reviews")

            csv_file = None
            for file in os.listdir(path):
                if file.endswith(".csv"):
                    csv_file = os.path.join(path, file)
                    break

            if not csv_file:
                msg = "No CSV file found in downloaded dataset"
                raise FileNotFoundError(msg)

            logger.info(f"Loading reviews from {csv_file}")
            df = pd.read_csv(csv_file)

            self._raw_data_cache = df.to_dict("records")
            logger.info(f"Loaded {len(self._raw_data_cache)} airline reviews from Kaggle dataset")
            return self._raw_data_cache

        except Exception as e:
            logger.exception(f"Error loading airline reviews from Kaggle: {e!s}")
            raise

    def process_to_texts(self):
        """Process raw data into formatted text strings for embedding (with caching)."""
        if self._processed_texts_cache is not None:
            return self._processed_texts_cache

        reviews = self.load_raw_data()
        review_texts = []

        for review in reviews:
            text_parts = []

            if review.get("AirLine_Name"):
                text_parts.append(f"Airline: {review['AirLine_Name']}")

            if review.get("Title"):
                text_parts.append(f"Title: {review['Title']}")

            if review.get("Review"):
                text_parts.append(f"Review: {review['Review']}")

            if review.get("Rating - 10"):
                text_parts.append(f"Rating: {review['Rating - 10']}/10")

            if review.get("Name"):
                text_parts.append(f"Reviewer: {review['Name']}")

            if review.get("Date"):
                text_parts.append(f"Date: {review['Date']}")

            if review.get("Recommond"):
                text_parts.append(f"Recommended: {review['Recommond']}")

            text = ". ".join(text_parts)
            review_texts.append(text)

        self._processed_texts_cache = review_texts
        logger.info(f"Processed {len(review_texts)} airline reviews into text format")
        return review_texts

    def load_to_vector_store(
        self,
        cluster,
        bucket_name: str,
        scope_name: str,
        collection_name: str,
        embeddings,
        index_name: str,
    ):
        """Load airline reviews into Couchbase vector store with embeddings."""
        try:
            count_query = (
                f"SELECT COUNT(*) as count FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            )
            count_result = cluster.query(count_query)
            count_row = next(iter(count_result))
            existing_count = count_row["count"]

            if existing_count > 0:
                logger.info(
                    f"Found {existing_count} existing documents in collection, skipping data load"
                )
                return

            review_texts = self.process_to_texts()

            vector_store = CouchbaseSearchVectorStore(
                cluster=cluster,
                bucket_name=bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embedding=embeddings,
                index_name=index_name,
            )

            logger.info(
                f"Loading {len(review_texts)} airline review embeddings to {bucket_name}.{scope_name}.{collection_name}"
            )

            batch_size = 10  # Conservative batch size for stability
            total_batches = (len(review_texts) + batch_size - 1) // batch_size

            with tqdm(
                total=len(review_texts), desc="Loading airline reviews", unit="reviews"
            ) as pbar:
                for i in range(0, len(review_texts), batch_size):
                    batch_num = i // batch_size + 1
                    batch = review_texts[i : i + batch_size]

                    vector_store.add_texts(texts=batch, batch_size=len(batch))

                    pbar.update(len(batch))
                    pbar.set_postfix(batch=f"{batch_num}/{total_batches}")

            logger.info(
                f"Successfully loaded {len(review_texts)} airline review embeddings to vector store"
            )

        except Exception as e:
            logger.exception(f"Error loading airline reviews to Couchbase: {e!s}")
            raise


# Global instance for reuse
_data_manager = AirlineReviewsDataManager()


def get_airline_review_texts():
    """Get processed airline review texts (uses global cached instance)."""
    return _data_manager.process_to_texts()


def load_airline_reviews_from_kaggle():
    """Load raw airline reviews data from Kaggle (uses global cached instance)."""
    return _data_manager.load_raw_data()


def load_airline_reviews_to_couchbase(
    cluster, bucket_name: str, scope_name: str, collection_name: str, embeddings, index_name: str
):
    """Load airline reviews into Couchbase vector store (uses global cached instance)."""
    return _data_manager.load_to_vector_store(
        cluster, bucket_name, scope_name, collection_name, embeddings, index_name
    )


def load_airline_reviews():
    """Simple function to load airline reviews - called by main.py."""
    try:
        # Just return the processed texts for embedding
        # This eliminates the need for separate cluster connection here
        logger.info("Loading airline reviews data...")
        reviews = _data_manager.process_to_texts()
        logger.info(f"Successfully loaded {len(reviews)} airline reviews")
        return reviews

    except Exception as e:
        logger.exception(f"Error in load_airline_reviews: {e!s}")
        raise

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


## Agent Classes and Core Implementation

Core agent classes and implementation from working main.py script.


In [21]:
# Agent classes are implemented using the inline AI services and Couchbase client above
print("All setup functions and client implementations are now available inline.")
def setup_ai_services(
    framework: str = "langchain",
    temperature: float = 0.0,
    callbacks: Optional[List] = None,
    application_span: Optional[Any] = None
) -> Tuple[Any, Any]:
    """Priority 1 AI service setup - Capella with direct API keys + OpenAI wrappers only."""
    embeddings = None
    llm = None

    logger.info(f"🔧 Setting up Priority 1 AI services for {framework} framework...")

    # PRIORITY 1: LATEST CAPELLA (OpenAI wrappers with direct API keys)
    if (
        not embeddings
        and os.getenv("CAPELLA_API_ENDPOINT")
        and os.getenv("CAPELLA_API_EMBEDDINGS_KEY")
    ):
        try:
            from langchain_openai import OpenAIEmbeddings
            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            api_key = os.getenv("CAPELLA_API_EMBEDDINGS_KEY")
            model = os.getenv("CAPELLA_API_EMBEDDING_MODEL")

            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                base_url = endpoint
            else:
                base_url = f"{endpoint}/v1"

            # Debug logging - same pattern as working test
            logger.info(f"🔧 Endpoint: {endpoint}")
            logger.info(f"🔧 Model: {model}")
            logger.info(f"🔧 Base URL: {base_url}")

            embeddings = OpenAIEmbeddings(
                model=model,
                api_key=api_key,
                base_url=base_url,
                check_embedding_ctx_length=False,  # KEY FIX for asymmetric models in LangChain/LangGraph
            )
            logger.info("✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI embeddings failed: {type(e).__name__}: {e}")

    if not llm and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_LLM_KEY"):
        try:
            from langchain_openai import ChatOpenAI

            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            llm_key = os.getenv("CAPELLA_API_LLM_KEY")
            llm_model = os.getenv("CAPELLA_API_LLM_MODEL")

            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                base_url = endpoint
            else:
                base_url = f"{endpoint}/v1"

            # Debug logging
            logger.info(f"🔧 LLM Endpoint: {endpoint}")
            logger.info(f"🔧 LLM Model: {llm_model}")
            logger.info(f"🔧 LLM Base URL: {base_url}")

            llm = ChatOpenAI(
                api_key=llm_key,
                base_url=base_url,
                model=llm_model,
                temperature=temperature,
            )
            # Test the LLM works
            from langchain_core.messages import HumanMessage
            test_response = llm.invoke([HumanMessage(content="Hello")])
            logger.info("✅ Using Priority 1: Capella AI LLM (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI LLM failed: {type(e).__name__}: {e}")
            llm = None

    # Fallback: OpenAI
    if not embeddings and os.getenv("OPENAI_API_KEY"):
        try:
            from langchain_openai import OpenAIEmbeddings
            embeddings = OpenAIEmbeddings(
                model="text-embedding-3-small",
                api_key=os.getenv("OPENAI_API_KEY"),
            )
            logger.info("✅ Using OpenAI embeddings fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI embeddings failed: {e}")

    if not llm and os.getenv("OPENAI_API_KEY"):
        try:
            from langchain_openai import ChatOpenAI
            llm = ChatOpenAI(
                api_key=os.getenv("OPENAI_API_KEY"),
                model="gpt-4o",
                temperature=temperature,
            )
            logger.info("✅ Using OpenAI LLM fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI LLM failed: {e}")

    if not embeddings:
        raise ValueError("❌ No embeddings service could be initialized")
    if not llm:
        raise ValueError("❌ No LLM service could be initialized")

    logger.info(f"✅ Priority 1 AI services setup completed for {framework}")
    return embeddings, llm


# Setup environment
setup_environment()

# Test Capella AI connectivity if configured
if os.getenv("CAPELLA_API_ENDPOINT"):
    if not test_capella_connectivity():
        logger.warning("❌ Capella AI connectivity test failed. Will use fallback models.")
else:
    logger.info("ℹ️ Capella API not configured - will use fallback models")

INFO:__main__:✅ Environment variables configured


All setup functions and client implementations are now available inline.


## CouchbaseClient Class

Define the CouchbaseClient for all database operations inline.


In [22]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=15))
            logger.info("✅ Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"❌ Failed to connect to Couchbase: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str, clear_existing_data: bool = False):
        try:
            if not self.cluster:
                self.connect()

            if not self.bucket:
                try:
                    self.bucket = self.cluster.bucket(self.bucket_name)
                except Exception:
                    logger.info(f"Creating bucket '{self.bucket_name}'...")
                    bucket_settings = CreateBucketSettings(
                        name=self.bucket_name, bucket_type=BucketType.COUCHBASE,
                        ram_quota_mb=1024, flush_enabled=True, num_replicas=0
                    )
                    self.cluster.buckets().create_bucket(bucket_settings)
                    time.sleep(5)
                    self.bucket = self.cluster.bucket(self.bucket_name)

            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                bucket_manager.create_scope(scope_name)

            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists and clear_existing_data:
                self.clear_collection_data(scope_name, collection_name)
            elif not collection_exists:
                bucket_manager.create_collection(scope_name, collection_name)

            time.sleep(3)
            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
            except Exception:
                pass

            collection = self.bucket.scope(scope_name).collection(collection_name)
            self._collections[f"{scope_name}.{collection_name}"] = collection
            logger.info(f"✅ Collection setup complete: {scope_name}.{collection_name}")
            return collection
        except Exception as e:
            raise RuntimeError(f"❌ Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str):
        try:
            logger.info(f"🗑️ Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")
            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)
            list(result)
            time.sleep(2)
        except KeyspaceNotFoundException:
            logger.info(f"ℹ️ Collection {self.bucket_name}.{scope_name}.{collection_name} doesn't exist, nothing to clear")
        except Exception as e:
            logger.warning(f"⚠️ Error clearing collection data: {e}")

    def clear_scope(self, scope_name: str):
        try:
            if not self.bucket:
                if not self.cluster:
                    self.connect()
                self.bucket = self.cluster.bucket(self.bucket_name)

            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            target_scope = next((s for s in scopes if s.name == scope_name), None)

            if target_scope:
                for collection in target_scope.collections:
                    try:
                        self.clear_collection_data(scope_name, collection.name)
                    except Exception:
                        pass
                logger.info(f"✅ Completed clearing scope: {self.bucket_name}.{scope_name}")
        except Exception as e:
            logger.warning(f"❌ Could not clear scope: {e}")

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        try:
            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"✅ Vector search index '{index_name}' created")
            else:
                logger.info(f"ℹ️ Vector search index '{index_name}' already exists")
        except Exception as e:
            logger.warning(f"⚠️ Error setting up vector search index: {e}")

    def setup_vector_store_langchain(self, scope_name: str, collection_name: str, index_name: str, embeddings, data_loader_func=None, **loader_kwargs):
        try:
            if data_loader_func:
                logger.info("🔄 Loading data into vector store...")
                data_loader_func(
                    cluster=self.cluster, bucket_name=self.bucket_name,
                    scope_name=scope_name, collection_name=collection_name,
                    embeddings=embeddings, index_name=index_name, **loader_kwargs
                )

            vector_store = CouchbaseSearchVectorStore(
                cluster=self.cluster, bucket_name=self.bucket_name,
                scope_name=scope_name, collection_name=collection_name,
                embedding=embeddings, index_name=index_name
            )
            logger.info(f"✅ Vector store setup complete: {self.bucket_name}.{scope_name}.{collection_name}")
            return vector_store
        except Exception as e:
            raise RuntimeError(f"❌ Error setting up vector store: {e!s}")


def create_couchbase_client():
    """Factory function to create CouchbaseClient with environment defaults."""
    return CouchbaseClient(
        conn_string=os.getenv("CB_CONN_STRING", "couchbase://localhost"),
        username=os.getenv("CB_USERNAME", "Administrator"),
        password=os.getenv("CB_PASSWORD", "password"),
        bucket_name=os.getenv("CB_BUCKET", "travel-sample"),
    )

## Flight Search Agent Classes

This cell contains the classes for the flight search agent.

### FlightSearchGraph

The `FlightSearchGraph` class is the main class for the flight search agent. It is a subclass of `langgraph.graph.StateGraph` and is used to define the graph of the flight search agent.

### FlightSearchGraphState

The `FlightSearchGraphState` class is the state of the flight search agent. It is a subclass of `langgraph.graph.State` and is used to define the state of the flight search agent.

In [23]:

## Agent Classes

class FlightSearchState(agentc_langgraph.agent.State):
    """State for flight search conversations - single user system."""

    query: str
    resolved: bool
    search_results: list[dict]


class FlightSearchAgent(agentc_langgraph.agent.ReActAgent):
    """Flight search agent using Agent Catalog tools and ReActAgent framework."""

    def __init__(self, catalog: agentc.Catalog, span: agentc.Span, chat_model=None):
        """Initialize the flight search agent."""

        if chat_model is None:
            # Fallback to OpenAI if no chat model provided
            model_name = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
            chat_model = langchain_openai.chat_models.ChatOpenAI(model=model_name, temperature=0.1)

        super().__init__(
            chat_model=chat_model, catalog=catalog, span=span, prompt_name="flight_search_assistant"
        )

    def _invoke(
        self,
        span: agentc.Span,
        state: FlightSearchState,
        config: langchain_core.runnables.RunnableConfig,
    ) -> FlightSearchState:
        """Handle flight search conversation using ReActAgent."""

        # Initialize conversation if this is the first message
        if not state["messages"]:
            initial_msg = langchain_core.messages.HumanMessage(content=state["query"])
            state["messages"].append(initial_msg)
            logger.info(f"Flight Query: {state['query']}")

        # Get prompt resource first - we'll need it for the ReAct agent
        prompt_resource = self.catalog.find("prompt", name="flight_search_assistant")

        # Get tools from Agent Catalog with simplified discovery
        tools = []
        tool_names = [
            "lookup_flight_info",
            "save_flight_booking",
            "retrieve_flight_bookings",
            "search_airline_reviews",
        ]

        for tool_name in tool_names:
            try:
                # Find tool using Agent Catalog
                catalog_tool = self.catalog.find("tool", name=tool_name)
                if catalog_tool:
                    logger.info(f"✅ Found tool: {tool_name}")
                else:
                    logger.error(f"❌ Tool not found: {tool_name}")
                    continue

            except Exception as e:
                logger.error(f"❌ Failed to find tool {tool_name}: {e}")
                continue

            # JSON-first architecture with Pydantic validation
            # Import validation schemas
            import json
            from pydantic import ValidationError
            from tools.schemas import FlightSearchInput, BookingInput

            def clean_react_artifacts(raw_input: str) -> str:
                """Remove ReAct format artifacts that contaminate tool inputs."""
                if not raw_input:
                    return ""

                cleaned = raw_input.strip()

                # Remove ReAct artifacts - order matters, check longer patterns first
                artifacts_to_remove = [
                    '\nObservation:', '\nObservation', 'Observation:', 'Observation',
                    '\nThought:', 'Thought:',
                    '\nAction:', 'Action:',
                    '\nAction Input:', 'Action Input:',
                    '\nFinal Answer:', 'Final Answer:',
                    'Observ'  # Handle incomplete artifact
                ]

                for artifact in artifacts_to_remove:
                    if artifact in cleaned:
                        # Split and take only the part before the artifact
                        cleaned = cleaned.split(artifact)[0].strip()

                # Clean up quotes and extra whitespace
                cleaned = cleaned.strip().strip("\"'").strip()

                # Normalize whitespace
                cleaned = " ".join(cleaned.split())

                return cleaned

            def parse_tool_input(tool_name: str, tool_input: str) -> dict:
                """Parse tool input: JSON with Pydantic validation for structured tools, plain text for simple tools."""
                # Clean ReAct artifacts first
                cleaned = clean_react_artifacts(tool_input)

                # Structured tools MUST use JSON
                if tool_name == "lookup_flight_info":
                    data = json.loads(cleaned)  # Will raise JSONDecodeError if invalid
                    validated = FlightSearchInput(**data)  # Will raise ValidationError if invalid
                    logger.info(f"✅ Parsed {tool_name}: {validated.source_airport} → {validated.destination_airport}")
                    return {
                        "source_airport": validated.source_airport,
                        "destination_airport": validated.destination_airport
                    }

                elif tool_name == "save_flight_booking":
                    data = json.loads(cleaned)  # Will raise JSONDecodeError if invalid
                    validated = BookingInput(**data)  # Will raise ValidationError if invalid
                    logger.info(f"✅ Parsed {tool_name}: {validated.source_airport}→{validated.destination_airport}, {validated.passengers} pax, {validated.flight_class}")
                    return {
                        "source_airport": validated.source_airport,
                        "destination_airport": validated.destination_airport,
                        "departure_date": validated.departure_date,
                        "passengers": validated.passengers,
                        "flight_class": validated.flight_class
                    }

                # Simple tools use plain text
                elif tool_name == "retrieve_flight_bookings":
                    return {"booking_query": cleaned}

                elif tool_name == "search_airline_reviews":
                    return {"query": cleaned}

                raise ValueError(f"Unknown tool: {tool_name}")

            # Create clean wrapper function for this tool
            def create_tool_func(catalog_tool_ref, tool_name_ref):
                """Create a wrapper that parses JSON and calls catalog tool with structured params."""
                def tool_func(tool_input: str) -> str:
                    try:
                        # Parse input with Pydantic validation (JSON for structured tools)
                        params = parse_tool_input(tool_name_ref, tool_input)

                        # Call the Agent Catalog tool with parsed parameters
                        result = catalog_tool_ref.func(**params)

                        return str(result) if result is not None else "No results found"

                    except Exception as e:
                        logger.error(f"❌ Error in tool {tool_name_ref}: {e}")
                        import traceback
                        logger.debug(traceback.format_exc())
                        return f"Error: {str(e)}"
                return tool_func

            # Tool descriptions for the LLM (JSON required for structured tools)
            tool_descriptions = {
                "lookup_flight_info": "Find flights between airports. REQUIRES JSON: {\"source_airport\": \"JFK\", \"destination_airport\": \"LAX\"}",
                "save_flight_booking": "Book a flight. REQUIRES JSON: {\"source_airport\": \"LAX\", \"destination_airport\": \"JFK\", \"departure_date\": \"2025-12-25\", \"passengers\": 2, \"flight_class\": \"business\"}",
                "retrieve_flight_bookings": "View all flight bookings or search by criteria. Leave input empty for all bookings.",
                "search_airline_reviews": "Search airline customer reviews. Input: plain text query (e.g., 'SpiceJet service quality')"
            }

            langchain_tool = Tool(
                name=tool_name,
                description=tool_descriptions.get(tool_name, f"Tool for {tool_name.replace('_', ' ')}"),
                func=create_tool_func(catalog_tool, tool_name),
            )
            tools.append(langchain_tool)

        # Use the Agent Catalog prompt content directly - get first result if it's a list
        if isinstance(prompt_resource, list):
            prompt_resource = prompt_resource[0]

        # Safely get the content from the prompt resource
        prompt_content = getattr(prompt_resource, "content", "")
        if not prompt_content:
            prompt_content = "You are a helpful flight search assistant. Use the available tools to help users with their flight queries."

        # Inject current date into the prompt content
        import datetime

        current_date = datetime.date.today().strftime("%Y-%m-%d")
        prompt_content = prompt_content.replace("{current_date}", current_date)

        # Use the Agent Catalog prompt content directly - it already has ReAct format
        react_prompt = PromptTemplate.from_template(str(prompt_content))

        # Create ReAct agent with tools and prompt
        agent = create_react_agent(self.chat_model, tools, react_prompt)

        # Custom parsing error handler - force stopping on parsing errors
        def handle_parsing_errors(error):
            """Custom handler for parsing errors - force early termination."""
            error_msg = str(error)
            if "both a final answer and a parse-able action" in error_msg:
                # Force early termination - return a reasonable response
                return "Final Answer: I encountered a parsing error. Please reformulate your request."
            elif "Missing 'Action:'" in error_msg:
                return "I need to use the correct format with Action: and Action Input:"
            else:
                return f"Final Answer: I encountered an error processing your request. Please try again."

        # Create agent executor - very strict: only 2 iterations max
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=handle_parsing_errors,
            max_iterations=2,  # STRICT: 1 tool call + 1 Final Answer only
            early_stopping_method="force",  # Force stop
            return_intermediate_steps=True,
        )

        # Execute the agent
        response = agent_executor.invoke({"input": state["query"]})

        # Extract tool outputs from intermediate_steps and store in search_results
        if "intermediate_steps" in response and response["intermediate_steps"]:
            tool_outputs = []
            for step in response["intermediate_steps"]:
                if isinstance(step, tuple) and len(step) >= 2:
                    # step[0] is the action, step[1] is the tool output/observation
                    tool_output = str(step[1])
                    if tool_output and tool_output.strip():
                        tool_outputs.append(tool_output)
            state["search_results"] = tool_outputs

        # Add response to conversation
        assistant_msg = langchain_core.messages.AIMessage(content=response["output"])
        state["messages"].append(assistant_msg)
        state["resolved"] = True

        return state


class FlightSearchGraph(agentc_langgraph.graph.GraphRunnable):
    """Flight search conversation graph using Agent Catalog."""

    def __init__(self, catalog, span, chat_model=None):
        """Initialize the flight search graph with optional chat model."""
        super().__init__(catalog=catalog, span=span)
        self.chat_model = chat_model

    @staticmethod
    def build_starting_state(query: str) -> FlightSearchState:
        """Build the initial state for the flight search - single user system."""
        return FlightSearchState(
            messages=[],
            query=query,
            resolved=False,
            search_results=[],
        )

    def compile(self):
        """Compile the LangGraph workflow."""

        # Build the flight search agent with catalog integration
        search_agent = FlightSearchAgent(
            catalog=self.catalog, span=self.span, chat_model=self.chat_model
        )

        # Create a wrapper function for the ReActAgent
        def flight_search_node(state: FlightSearchState) -> FlightSearchState:
            """Wrapper function for the flight search ReActAgent."""
            return search_agent._invoke(
                span=self.span,
                state=state,
                config={},  # Empty config for now
            )

        # Create a simple workflow graph for flight search
        workflow = langgraph.graph.StateGraph(FlightSearchState)

        # Add the flight search agent node using the wrapper function
        workflow.add_node("flight_search", flight_search_node)

        # Set entry point and simple flow
        workflow.set_entry_point("flight_search")
        workflow.add_edge("flight_search", langgraph.graph.END)

        return workflow.compile()


## Data Loading Components

Complete data loading and query definition components embedded inline for standalone operation.


## Clear Existing Data

Clear existing bookings and reviews for clean test run.


In [24]:
def clear_bookings_and_reviews():
    """Clear existing flight bookings to start fresh for demo."""
    try:
        client = create_couchbase_client()
        client.connect()

        # Clear bookings scope using environment variables
        bookings_scope = "agentc_bookings"
        client.clear_scope(bookings_scope)
        logger.info(
            f"✅ Cleared existing flight bookings for fresh test run: {os.environ['CB_BUCKET']}.{bookings_scope}"
        )

        # Check if airline reviews collection needs clearing by comparing expected vs actual document count
        try:
            # Get expected document count (this uses cached data if available)
            expected_docs = _data_manager.process_to_texts()
            expected_count = len(expected_docs)

            # Check current document count in collection
            try:
                count_query = f"SELECT COUNT(*) as count FROM `{os.environ['CB_BUCKET']}`.`{os.environ['CB_SCOPE']}`.`{os.environ['CB_COLLECTION']}`"
                count_result = client.cluster.query(count_query)
                count_row = next(iter(count_result))
                existing_count = count_row["count"]

                logger.info(
                    f"📊 Airline reviews collection: {existing_count} existing, {expected_count} expected"
                )

                if existing_count == expected_count:
                    logger.info(
                        f"✅ Collection already has correct document count ({existing_count}), skipping clear"
                    )
                else:
                    logger.info(
                        f"🗑️  Clearing airline reviews collection: {os.environ['CB_BUCKET']}.{os.environ['CB_SCOPE']}.{os.environ['CB_COLLECTION']}"
                    )
                    client.clear_collection_data(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"])
                    logger.info(
                        f"✅ Cleared existing airline reviews for fresh data load: {os.environ['CB_BUCKET']}.{os.environ['CB_SCOPE']}.{os.environ['CB_COLLECTION']}"
                    )

            except KeyspaceNotFoundException:
                # Collection doesn't exist yet - this is expected for fresh setup
                logger.info(
                    f"📊 Collection doesn't exist yet, will create and load fresh data"
                )
            except Exception as count_error:
                # Other query errors - clear anyway to ensure fresh start
                logger.info(
                    f"📊 Collection query failed, will clear and reload: {count_error}"
                )
                client.clear_collection_data(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"])
                logger.info(
                    f"✅ Cleared existing airline reviews for fresh data load: {os.environ['CB_BUCKET']}.{os.environ['CB_SCOPE']}.{os.environ['CB_COLLECTION']}"
                )

        except Exception as e:
            logger.warning(f"⚠️  Could not check collection count, clearing anyway: {e}")
            client.clear_collection_data(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"])
            logger.info(
                f"✅ Cleared existing airline reviews for fresh data load: {os.environ['CB_BUCKET']}.{os.environ['CB_SCOPE']}.{os.environ['CB_COLLECTION']}"
            )

    except Exception as e:
        logger.warning(f"❌ Could not clear bookings: {e}")


# Clear existing data for fresh test run
clear_bookings_and_reviews()


INFO:__main__:✅ Successfully connected to Couchbase
INFO:__main__:🗑️ Clearing data from travel-sample.agentc_bookings.user_bookings_20251023...
INFO:__main__:🗑️ Clearing data from travel-sample.agentc_bookings.user_bookings_20251022...
INFO:__main__:✅ Completed clearing scope: travel-sample.agentc_bookings
INFO:__main__:✅ Cleared existing flight bookings for fresh test run: travel-sample.agentc_bookings
INFO:__main__:Downloading Indian Airlines Customer Reviews dataset from Kaggle...


Using Colab cache for faster access to the 'indian-airlines-customer-reviews' dataset.


INFO:__main__:Loading reviews from /kaggle/input/indian-airlines-customer-reviews/Indian_Domestic_Airline.csv
INFO:__main__:Loaded 2210 airline reviews from Kaggle dataset
INFO:__main__:Processed 2210 airline reviews into text format
INFO:__main__:📊 Airline reviews collection: 2210 existing, 2210 expected
INFO:__main__:✅ Collection already has correct document count (2210), skipping clear


## Setup Flight Search Agent

Initialize the complete flight search agent setup using the refactored approach.


In [25]:
def setup_flight_search_agent():
    """Common setup function for flight search agent - returns all necessary components."""
    try:
        # Setup environment first
        setup_environment()

        # Initialize Agent Catalog (uses .env or defaults if AGENT_CATALOG_* vars not set)
        catalog = agentc.Catalog()
        application_span = catalog.Span(name="Flight Search Agent", blacklist=set())

        # Test Capella AI connectivity
        if os.getenv("CAPELLA_API_ENDPOINT"):
            if not test_capella_connectivity():
                logger.warning("❌ Capella AI connectivity test failed. Will use fallback models.")
        else:
            logger.info("ℹ️ Capella API not configured - will use fallback models")

        # Create CouchbaseClient for all operations
        client = create_couchbase_client()

        # Setup everything in one call - bucket, scope, collection
        client.setup_collection(
            scope_name=os.environ["CB_SCOPE"],
            collection_name=os.environ["CB_COLLECTION"],
            clear_existing_data=False,  # Let data loader decide based on count check
        )

        # Setup vector search index
        try:
            with open("agentcatalog_index.json") as file:
                index_definition = json.load(file)
            logger.info("Loaded vector search index definition from agentcatalog_index.json")
            client.setup_vector_search_index(index_definition, os.environ["CB_SCOPE"])
        except Exception as e:
            logger.warning(f"Error loading index definition: {e!s}")
            logger.info("Continuing without vector search index...")


        # Setup AI services using Priority 1: Capella AI + OpenAI wrappers
        embeddings, _ = setup_ai_services(framework="langgraph")

        # Setup vector store with airline reviews data
        vector_store = client.setup_vector_store_langchain(
            scope_name=os.environ["CB_SCOPE"],
            collection_name=os.environ["CB_COLLECTION"],
            index_name=os.environ["CB_INDEX"],
            embeddings=embeddings,
            data_loader_func=load_airline_reviews_to_couchbase,
        )

        # Setup LLM using Priority 1: Capella AI + OpenAI wrappers
        _, chat_model = setup_ai_services(framework="langgraph", temperature=0.1)

        # Create the flight search graph with the chat model
        flight_graph = FlightSearchGraph(
            catalog=catalog, span=application_span, chat_model=chat_model
        )
        # Compile the graph
        compiled_graph = flight_graph.compile()

        logger.info("Agent Catalog integration successful")

        return compiled_graph, application_span

    except Exception as e:
        logger.exception(f"Setup error: {e}")
        logger.info("Ensure Agent Catalog is published: agentc index . && agentc publish")
        raise


def run_test_query(test_number: int, query: str, compiled_graph, application_span):
    """Run a single test query with error handling."""
    logger.info(f"\n🔍 Test {test_number}: {query}")
    try:
        state = FlightSearchGraph.build_starting_state(query=query)
        result = compiled_graph.invoke(state)

        if result.get("search_results"):
            logger.info(f"Found {len(result['search_results'])} flight options")
        logger.info(f"✅ Test {test_number} completed: {result.get('resolved', False)}")

        return result

    except Exception as e:
        logger.exception(f"❌ Test {test_number} failed: {e}")
        return None


# Setup the agent
compiled_graph, application_span = setup_flight_search_agent()

INFO:__main__:✅ Environment variables configured
INFO:__main__:✅ Successfully connected to Couchbase
INFO:__main__:✅ Collection setup complete: agentc_data.airline_reviews
INFO:__main__:Loaded vector search index definition from agentcatalog_index.json
INFO:__main__:ℹ️ Vector search index 'airline_reviews_index' already exists
INFO:__main__:🔧 Setting up Priority 1 AI services for langgraph framework...
INFO:__main__:🔧 Endpoint: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com
INFO:__main__:🔧 Model: nvidia/llama-3.2-nv-embedqa-1b-v2
INFO:__main__:🔧 Base URL: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com/v1
INFO:__main__:✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)
INFO:__main__:🔧 LLM Endpoint: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com
INFO:__main__:🔧 LLM Model: meta/llama3-8b-instruct
INFO:__main__:🔧 LLM Base URL: https://agd6zdjymyanhi9g.ai.sandbox.nonprod-project-avengers.com/v1
INFO:__main__:✅ Using Priority 1: C

## Test 1: Flight Search

Find flights from JFK to LAX for tomorrow.


In [26]:
result1 = run_test_query(
    1, "Find flights from JFK to LAX for tomorrow", compiled_graph, application_span
)

INFO:__main__:
🔍 Test 1: Find flights from JFK to LAX for tomorrow
INFO:__main__:Flight Query: Find flights from JFK to LAX for tomorrow
INFO:__main__:✅ Found tool: lookup_flight_info
INFO:__main__:✅ Found tool: save_flight_booking
INFO:__main__:✅ Found tool: retrieve_flight_bookings
INFO:__main__:✅ Found tool: search_airline_reviews




> Entering new AgentExecutor chain...


INFO:__main__:✅ Parsed lookup_flight_info: JFK → LAX
INFO:lookup_flight_info:🔍 Looking up flights: JFK → LAX


Thought: I need to use lookup_flight_info to help with this request Action: lookup_flight_info Action Input: {"source_airport": "JFK", "destination_airport": "LAX"}Available flights from JFK to LAX:

1. AS flight from JFK to LAX using 321 762
2. B6 flight from JFK to LAX using 320
3. DL flight from JFK to LAX using 76W 752
4. QF flight from JFK to LAX using 744
5. AA flight from JFK to LAX using 32B 762
6. UA flight from JFK to LAX using 757
7. US flight from JFK to LAX using 32B 762
8. VX flight from JFK to LAX using 320

INFO:__main__:Found 1 flight options
INFO:__main__:✅ Test 1 completed: True


Thought: I now know the final answer
Final Answer: I found 8 available flights from JFK to LAX, including airlines AS, B6, DL, QF, AA, UA, US, and VX with various aircraft types.

> Finished chain.


## Test 2: Flight Booking (Business Class)

Book a flight with business class.


In [27]:
result2 = run_test_query(
    2,
    "Book a flight from LAX to JFK for tomorrow, 2 passengers, business class",
    compiled_graph,
    application_span,
)

INFO:__main__:
🔍 Test 2: Book a flight from LAX to JFK for tomorrow, 2 passengers, business class
INFO:__main__:Flight Query: Book a flight from LAX to JFK for tomorrow, 2 passengers, business class
INFO:__main__:✅ Found tool: lookup_flight_info
INFO:__main__:✅ Found tool: save_flight_booking
INFO:__main__:✅ Found tool: retrieve_flight_bookings
INFO:__main__:✅ Found tool: search_airline_reviews




> Entering new AgentExecutor chain...


INFO:__main__:✅ Parsed save_flight_booking: LAX→JFK, 2 pax, business
INFO:save_flight_booking:🗓️ Date validated: 2025-10-24


Thought: I need to use save_flight_booking to help with this request. Tomorrow is 2025-10-23 + 1 day = 2025-10-24 Action: save_flight_booking Action Input: {"source_airport": "LAX", "destination_airport": "JFK", "departure_date": "2025-10-24", "passengers": 2, "flight_class": "business"}

INFO:save_flight_booking:🎯 Booking: LAX→JFK on 2025-10-24, 2 pax, business class


Flight Booking Confirmed!

Booking ID: FL1024BDE6EE8C
Route: LAX → JFK
Departure Date: 2025-10-24
Passengers: 2
Class: business
Total Price: $1500.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID

Thank you for choosing our airline!

INFO:__main__:Found 1 flight options
INFO:__main__:✅ Test 2 completed: True


Thought: I now know the final answer Final Answer: Your flight from LAX to JFK for 2 passengers in business class has been booked successfully for tomorrow (2025-10-24). Your booking ID is FL1024BDE6EE8C and the total cost is $1500.00.

> Finished chain.


## Test 3: Flight Booking (Economy Class)

Book an economy flight.


In [28]:
result3 = run_test_query(
    3,
    "Book an economy flight from JFK to MIA for next week, 1 passenger",
    compiled_graph,
    application_span,
)

INFO:__main__:
🔍 Test 3: Book an economy flight from JFK to MIA for next week, 1 passenger
INFO:__main__:Flight Query: Book an economy flight from JFK to MIA for next week, 1 passenger
INFO:__main__:✅ Found tool: lookup_flight_info
INFO:__main__:✅ Found tool: save_flight_booking
INFO:__main__:✅ Found tool: retrieve_flight_bookings
INFO:__main__:✅ Found tool: search_airline_reviews




> Entering new AgentExecutor chain...


INFO:__main__:✅ Parsed save_flight_booking: JFK→MIA, 1 pax, economy
INFO:save_flight_booking:🗓️ Date validated: 2025-10-30


Thought: I need to use save_flight_booking to help with this request. Next week is 2025-10-23 + 7 days = 2025-10-30 Action: save_flight_booking Action Input: {"source_airport": "JFK", "destination_airport": "MIA", "departure_date": "2025-10-30", "passengers": 1, "flight_class": "economy"}

INFO:save_flight_booking:🎯 Booking: JFK→MIA on 2025-10-30, 1 pax, economy class


Flight Booking Confirmed!

Booking ID: FL10306C68E47C
Route: JFK → MIA
Departure Date: 2025-10-30
Passengers: 1
Class: economy
Total Price: $250.00

Next Steps:
1. Check-in opens 24 hours before departure
2. Arrive at airport 2 hours early for domestic flights
3. Bring valid government-issued photo ID

Thank you for choosing our airline!

INFO:__main__:Found 1 flight options
INFO:__main__:✅ Test 3 completed: True


Thought: I now know the final answer
Final Answer: Your economy flight from JFK to MIA for 1 passenger has been booked successfully for next week (2025-10-30). Your booking ID is FL10306C68E47C and the total cost is $250.00.

> Finished chain.


## Test 4: Retrieve Current Bookings

Show current flight bookings.


In [29]:
result4 = run_test_query(4, "Show me my current flight bookings", compiled_graph, application_span)

INFO:__main__:
🔍 Test 4: Show me my current flight bookings
INFO:__main__:Flight Query: Show me my current flight bookings
INFO:__main__:✅ Found tool: lookup_flight_info
INFO:__main__:✅ Found tool: save_flight_booking
INFO:__main__:✅ Found tool: retrieve_flight_bookings
INFO:__main__:✅ Found tool: search_airline_reviews




> Entering new AgentExecutor chain...
Thought: I need to use retrieve_flight_bookings to help with this request Action: retrieve_flight_bookings Action Input:Your Current Bookings (2 found):

Booking 1:
  Booking ID: FL10306C68E47C
  Route: JFK → MIA
  Date: 2025-10-30
  Passengers: 1
  Class: economy
  Total: $250.00
  Status: confirmed
  Booked: 2025-10-23

Booking 2:
  Booking ID: FL1024BDE6EE8C
  Route: LAX → JFK
  Date: 2025-10-24
  Passengers: 2
  Class: business
  Total: $1500.00
  Status: confirmed
  Booked: 2025-10-23

INFO:__main__:Found 1 flight options
INFO:__main__:✅ Test 4 completed: True


Thought: I now know the final answer
Final Answer: You have 2 current flight bookings: (1) JFK to MIA on 2025-10-30 for 1 passenger in economy class ($250.00), and (2) LAX to JFK on 2025-10-24 for 2 passengers in business class ($1500.00).

> Finished chain.


## Test 5: Airline Reviews Search

Search airline reviews for service quality.


In [30]:
result5 = run_test_query(
    5, "What do passengers say about SpiceJet's service quality?", compiled_graph, application_span
)

INFO:__main__:
🔍 Test 5: What do passengers say about SpiceJet's service quality?
INFO:__main__:Flight Query: What do passengers say about SpiceJet's service quality?
INFO:__main__:✅ Found tool: lookup_flight_info
INFO:__main__:✅ Found tool: save_flight_booking
INFO:__main__:✅ Found tool: retrieve_flight_bookings
INFO:__main__:✅ Found tool: search_airline_reviews




> Entering new AgentExecutor chain...
Thought: I need to use search_airline_reviews to help with this request Action: search_airline_reviews Action Input: SpiceJet service quality

INFO:search_airline_reviews:Searching for airline reviews with query: 'SpiceJet service quality'
INFO:search_airline_reviews:Found 5 results for query: 'SpiceJet service quality'


Found 5 relevant airline reviews for 'SpiceJet service quality':

Review 1:
Airline: SpiceJet. Title: "Great travel experience". Review: ✅ Trip Verified |  Marvelous courteous crew who took good care of all passengers. They should be rewarded for the patience shown towards the unruly ones. Great travel experience.. Rating: 10.0/10. Reviewer: Ranjita Pandey. Date: 18th April 2024. Recommended: yes

Review 2:
Airline: SpiceJet. Title: "good service by the crew". Review: ✅ Trip Verified | I have had good service by the crew. It was amazing, the crew was very enthusiastic and warm welcome. It was one of the best services in my experience.. Rating: 10.0/10. Reviewer: K Mansour. Date: 10th August 2024. Recommended: yes

Review 3:
Airline: SpiceJet. Title: "Excellent service". Review: ✅ Trip Verified |  Excellent service by the ground staff courteous beyond expectations always willing to help in the real sense and not lipservice i will recommend to all whom I know. Rating: 10.0/10. Reviewer: 

INFO:__main__:Found 1 flight options
INFO:__main__:✅ Test 5 completed: True


Thought: I now know the final answer Final Answer: Based on 5 customer reviews, SpiceJet receives mostly positive feedback (averaging 8.4/10) for their courteous crew, warm hospitality, and excellent ground staff service. However, some passengers report flight delays and poor service. The majority of reviews are highly positive (four 10/10 ratings), with one negative review citing delays and lack of amenities.

> Finished chain.


## Arize Phoenix Evaluation

This section demonstrates how to evaluate the flight search agent using Arize Phoenix observability platform. The evaluation includes:

- **Relevance Scoring**: Using Phoenix RelevanceEvaluator to score how relevant responses are to queries
- **QA Scoring**: Using Phoenix QAEvaluator with lenient evaluation templates for better accuracy
- **Hallucination Detection**: Using Phoenix HallucinationEvaluator with lenient templates to detect fabricated information
- **Toxicity Detection**: Using Phoenix ToxicityEvaluator to detect harmful content
- **Phoenix UI**: Real-time observability dashboard

We'll run evaluation queries and assess the responses for quality and safety using the latest evaluation approach.


## Evaluation Configuration

Configuration class for the evaluation system with all parameters.


In [31]:
import pandas as pd
import nest_asyncio
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Any

# Apply the patch to allow nested asyncio event loops
nest_asyncio.apply()

# Try to import Arize dependencies with fallback
try:
    import phoenix as px
    from arize.experimental.datasets import ArizeDatasetsClient
    from arize.experimental.datasets.utils.constants import GENERATIVE
    from openinference.instrumentation.langchain import LangChainInstrumentor
    from openinference.instrumentation.openai import OpenAIInstrumentor
    from phoenix.evals import (
        RAG_RELEVANCY_PROMPT_RAILS_MAP,
        RAG_RELEVANCY_PROMPT_TEMPLATE,
        TOXICITY_PROMPT_RAILS_MAP,
        TOXICITY_PROMPT_TEMPLATE,
        HallucinationEvaluator,
        OpenAIModel,
        QAEvaluator,
        RelevanceEvaluator,
        ToxicityEvaluator,
        llm_classify,
    )
    from phoenix.otel import register

    ARIZE_AVAILABLE = True
except ImportError as e:
    logger.warning(f"Arize dependencies not available: {e}")
    logger.warning("Running in local evaluation mode only...")
    ARIZE_AVAILABLE = False

# Define lenient evaluation templates for flight search agent evaluation
LENIENT_QA_PROMPT_TEMPLATE = """
You are evaluating whether an AI agent's response correctly addresses a user's question.

FOCUS ON FUNCTIONAL SUCCESS, NOT EXACT MATCHING:
1. Did the agent provide the requested information (flights, bookings, reviews)?
2. Is the core information accurate and helpful to the user?
3. Would the user be satisfied with what they received?

DYNAMIC DATA IS EXPECTED AND CORRECT:
- Booking IDs will be DIFFERENT each time (dynamically generated - this is correct!)
- Dates like "tomorrow" are calculated dynamically (may differ from reference)
- Booking lists reflect ACTUAL session bookings (may differ from reference)
- Route sequences depend on actual booking order in this session

IGNORE THESE DIFFERENCES:
- Different booking IDs, dates, or sequences (these are dynamic!)
- Format differences, duplicate calls, system messages
- Reference mismatches due to dynamic data

MARK AS CORRECT IF:
- Agent successfully completed the action (found flights, made booking, retrieved bookings, got reviews)
- User received useful, accurate information
- Core functionality worked as expected

Question: {input}
Reference Answer: {reference}
Agent Response: {output}

Did the agent successfully provide what the user requested, regardless of exact reference matching?
Respond with just "correct" or "incorrect".
"""

LENIENT_HALLUCINATION_PROMPT_TEMPLATE = """
You are checking if an AI agent's response contains hallucinated information.

DYNAMIC DATA IS EXPECTED AND FACTUAL:
- Booking IDs are dynamically generated (will ALWAYS be different from reference - this is correct!)
- Dates are calculated dynamically ("tomorrow", "next week" based on current date)
- Booking sequences reflect actual session bookings (not static reference data)
- Tool outputs contain real system data

MARK AS FACTUAL IF:
- Response contains "iteration limit" or "time limit" (system issue, not hallucination)
- Dynamic data differs from reference (booking IDs, dates, booking sequences)
- Agent provides plausible flight data, booking confirmations, or reviews
- Information is consistent with system capabilities

ONLY MARK AS HALLUCINATED IF:
- Response contains clearly impossible information (fake airlines, impossible routes)
- Agent makes up data it cannot access
- Response contradicts fundamental system facts

REMEMBER: Different booking IDs, dates, and sequences are EXPECTED dynamic behavior!

Question: {input}
Reference Text: {reference}
Agent Response: {output}

Does the response contain clearly false information, ignoring expected dynamic data differences?
Respond with just "factual" or "hallucinated".
"""

# Custom Rails for lenient evaluation
LENIENT_QA_RAILS = ["correct", "incorrect"]
LENIENT_HALLUCINATION_RAILS = ["factual", "hallucinated"]


@dataclass
class EvaluationConfig:
    """Configuration for the evaluation system."""

    # Arize Configuration
    arize_space_id: str = os.getenv("ARIZE_SPACE_ID", "your-space-id")
    arize_api_key: str = os.getenv("ARIZE_API_KEY", "your-api-key")
    project_name: str = "flight-search-agent-evaluation"

    # Phoenix Configuration
    phoenix_base_port: int = 6006
    phoenix_grpc_base_port: int = 4317
    phoenix_max_port_attempts: int = 5
    phoenix_startup_timeout: int = 30

    # Evaluation Configuration
    evaluator_model: str = "gpt-4o"
    batch_size: int = 10
    max_retries: int = 3
    evaluation_timeout: int = 300

    # Logging Configuration
    log_level: str = "INFO"
    detailed_logging: bool = True

    # Dataset Configuration
    dataset_name: str = "flight-search-evaluation-queries"
    dataset_description: str = "Flight search agent evaluation queries with expected responses"

    # Output Configuration
    save_results: bool = True
    results_format: str = "csv"  # csv, json, both
    output_dir: str = "output"

    def __post_init__(self):
        """Validate configuration after initialization."""
        if self.phoenix_base_port < 1024 or self.phoenix_base_port > 65535:
            raise ValueError("Phoenix base port must be between 1024 and 65535")

        if self.batch_size < 1 or self.batch_size > 100:
            raise ValueError("Batch size must be between 1 and 100")

        if self.evaluation_timeout < 30:
            raise ValueError("Evaluation timeout must be at least 30 seconds")


INFO:phoenix.config:📋 Ensuring phoenix working directory: /root/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_f904ec33-2873-40e1-ae8f-40e7e45643e5 initialized


## Phoenix Manager

Manages Phoenix server lifecycle and port management.


In [32]:
import socket
import subprocess
import time

class PhoenixManager:
    """Manages Phoenix server lifecycle and port management."""

    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.session = None
        self.active_port = None
        self.tracer_provider = None

    def _is_port_in_use(self, port: int) -> bool:
        """Check if a port is in use."""
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            return s.connect_ex(("localhost", port)) == 0

    def _kill_existing_phoenix_processes(self) -> None:
        """Kill any existing Phoenix processes."""
        try:
            subprocess.run(["pkill", "-f", "phoenix"], check=False, capture_output=True)
            time.sleep(2)  # Wait for processes to terminate
        except Exception as e:
            logger.debug(f"Error killing Phoenix processes: {e}")

    def _find_available_port(self) -> Tuple[int, int]:
        """Find available ports for Phoenix."""
        phoenix_port = self.config.phoenix_base_port
        grpc_port = self.config.phoenix_grpc_base_port

        for _ in range(self.config.phoenix_max_port_attempts):
            if not self._is_port_in_use(phoenix_port):
                return phoenix_port, grpc_port
            phoenix_port += 1
            grpc_port += 1

        raise RuntimeError(
            f"Could not find available ports after {self.config.phoenix_max_port_attempts} attempts"
        )

    def start_phoenix(self) -> bool:
        """Start Phoenix server and return success status."""
        if not ARIZE_AVAILABLE:
            logger.warning("⚠️ Phoenix dependencies not available")
            return False

        try:
            logger.info("🔧 Setting up Phoenix observability...")

            # Clean up existing processes
            self._kill_existing_phoenix_processes()

            # Find available ports
            phoenix_port, grpc_port = self._find_available_port()

            # Set environment variables
            os.environ["PHOENIX_PORT"] = str(phoenix_port)
            os.environ["PHOENIX_GRPC_PORT"] = str(grpc_port)

            # Start Phoenix session
            self.session = px.launch_app()
            self.active_port = phoenix_port

            if self.session:
                logger.info(f"🌐 Phoenix UI: {self.session.url}")

            # Register Phoenix OTEL
            self.tracer_provider = register(
                project_name=self.config.project_name,
                endpoint=f"http://localhost:{phoenix_port}/v1/traces",
            )

            logger.info("✅ Phoenix setup completed successfully")
            return True

        except Exception as e:
            logger.exception(f"❌ Phoenix setup failed: {e}")
            return False

    def setup_instrumentation(self) -> bool:
        """Setup OpenTelemetry instrumentation."""
        if not self.tracer_provider or not ARIZE_AVAILABLE:
            return False

        try:
            instrumentors = [
                ("LangChain", LangChainInstrumentor),
                ("OpenAI", OpenAIInstrumentor),
            ]

            for name, instrumentor_class in instrumentors:
                try:
                    instrumentor = instrumentor_class()
                    instrumentor.instrument(tracer_provider=self.tracer_provider)
                    logger.info(f"✅ {name} instrumentation enabled")
                except Exception as e:
                    logger.warning(f"⚠️ {name} instrumentation failed: {e}")

            return True

        except Exception as e:
            logger.exception(f"❌ Instrumentation setup failed: {e}")
            return False

    def cleanup(self) -> None:
        """Clean up Phoenix resources."""
        try:
            # Clean up environment variables
            for var in ["PHOENIX_PORT", "PHOENIX_GRPC_PORT"]:
                if var in os.environ:
                    del os.environ[var]

            logger.info("🔒 Phoenix cleanup completed")
        except Exception as e:
            logger.warning(f"⚠️ Error during Phoenix cleanup: {e}")


## Arize Dataset Manager

Manages Arize dataset creation and management.


In [33]:
from datetime import datetime

class ArizeDatasetManager:
    """Manages Arize dataset creation and management."""

    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.client = None
        self._setup_client()

    def _setup_client(self) -> None:
        """Setup Arize datasets client."""
        if not ARIZE_AVAILABLE:
            return

        if (
            self.config.arize_api_key != "your-api-key"
            and self.config.arize_space_id != "your-space-id"
        ):
            try:
                # Initialize with correct parameters - no space_id needed for datasets client
                self.client = ArizeDatasetsClient(
                    api_key=self.config.arize_api_key
                )
                logger.info("✅ Arize datasets client initialized successfully")
            except Exception as e:
                logger.warning(f"⚠️ Could not initialize Arize datasets client: {e}")
                self.client = None
        else:
            logger.warning("⚠️ Arize API credentials not configured")
            self.client = None

    def create_dataset(self, results_df: pd.DataFrame) -> Optional[str]:
        """Create Arize dataset from evaluation results."""
        if not self.client:
            logger.warning("⚠️ Arize client not available - skipping dataset creation")
            return None

        try:
            dataset_name = f"flight-search-evaluation-{datetime.now().strftime('%Y%m%d_%H%M%S')}"

            logger.info("📊 Creating Arize dataset...")
            dataset_id = self.client.create_dataset(
                space_id=self.config.arize_space_id,
                dataset_name=dataset_name,
                dataset_type=GENERATIVE,
                data=results_df,
                convert_dict_to_json=True
            )

            if dataset_id:
                logger.info(f"✅ Arize dataset created successfully: {dataset_id}")
                return dataset_id
            else:
                logger.warning("⚠️ Dataset creation returned None")
                return None

        except Exception as e:
            logger.error(f"❌ Error creating Arize dataset: {e}")
            return None

## Arize Flight Search Evaluator

Main evaluator class for comprehensive flight search agent evaluation.


In [34]:
class ArizeFlightSearchEvaluator:
    """
    Streamlined flight search agent evaluator using only Arize Phoenix evaluators.

    This class provides comprehensive evaluation capabilities using:
    - Phoenix RelevanceEvaluator for response relevance
    - Phoenix QAEvaluator for correctness assessment
    - Phoenix HallucinationEvaluator for factual accuracy
    - Phoenix ToxicityEvaluator for safety assessment
    - No manual validation - Phoenix evaluators only
    """

    def __init__(self, config: Optional[EvaluationConfig] = None):
        """Initialize the evaluator with configuration."""
        self.config = config or EvaluationConfig()
        self._setup_logging()

        # Initialize components
        self.phoenix_manager = PhoenixManager(self.config)
        self.dataset_manager = ArizeDatasetManager(self.config)

        # Agent components
        self.agent = None
        self.span = None

        # Phoenix evaluators
        self.evaluators = {}
        self.evaluator_llm = None

        if ARIZE_AVAILABLE:
            self._setup_phoenix_evaluators()

    def _setup_logging(self) -> None:
        """Configure logging to suppress verbose modules."""
        verbose_modules = ["openai", "httpx", "httpcore", "agentc_core"]
        for module in verbose_modules:
            logging.getLogger(module).setLevel(logging.WARNING)

    def _setup_phoenix_evaluators(self) -> None:
        """Setup Phoenix evaluators with robust error handling."""
        if not ARIZE_AVAILABLE:
            logger.warning("⚠️ Phoenix dependencies not available - evaluations will be limited")
            return

        try:
            self.evaluator_llm = OpenAIModel(model=self.config.evaluator_model)

            # Initialize all Phoenix evaluators
            self.evaluators = {
                "relevance": RelevanceEvaluator(self.evaluator_llm),
                "qa_correctness": QAEvaluator(self.evaluator_llm),
                "hallucination": HallucinationEvaluator(self.evaluator_llm),
                "toxicity": ToxicityEvaluator(self.evaluator_llm),
            }

            logger.info("✅ Phoenix evaluators initialized successfully")
            logger.info(f"   🤖 Using evaluator model: {self.config.evaluator_model}")
            logger.info(f"   📊 Available evaluators: {list(self.evaluators.keys())}")

            # Setup Phoenix if available
            if self.phoenix_manager.start_phoenix():
                self.phoenix_manager.setup_instrumentation()

        except Exception as e:
            logger.warning(f"⚠️ Phoenix evaluators setup failed: {e}")
            logger.info("Continuing with basic evaluation metrics only...")
            self.evaluators = {}

    def setup_agent(self) -> bool:
        """Setup flight search agent using refactored main.py setup."""
        try:
            logger.info("🔧 Setting up flight search agent...")

            # Use the refactored setup function from main.py
            compiled_graph, application_span = setup_flight_search_agent()

            self.agent = compiled_graph
            self.span = application_span

            logger.info("✅ Flight search agent setup completed successfully")
            return True

        except Exception as e:
            logger.exception(f"❌ Error setting up flight search agent: {e}")
            return False

    def _extract_response_content(self, result: Any) -> str:
        """Extract complete response content including tool results from agent result."""
        try:
            response_parts = []

            # Critical Fix: Extract tool outputs from search_results first
            if isinstance(result, dict) and "search_results" in result:
                search_results = result["search_results"]
                if search_results:
                    # search_results contains the actual tool outputs we want
                    response_parts.append(str(search_results))

            # Also check for intermediate_steps (AgentExecutor format)
            if isinstance(result, dict) and "intermediate_steps" in result:
                for step in result["intermediate_steps"]:
                    if isinstance(step, tuple) and len(step) >= 2:
                        # step[1] is the tool output/observation
                        tool_output = str(step[1])
                        if tool_output and tool_output.strip():
                            response_parts.append(tool_output)

            # Check for messages from LangGraph state (but filter out generic ones)
            if hasattr(result, "messages") and result.messages:
                for message in result.messages:
                    if hasattr(message, "content") and message.content:
                        content = str(message.content)
                        # Skip generic system messages and human messages
                        if (hasattr(message, "type") and message.type != "human" and
                            not any(phrase in content.lower() for phrase in
                                   ["iteration limit", "time limit", "agent stopped"])):
                            response_parts.append(content)
            elif isinstance(result, dict) and "messages" in result:
                for message in result["messages"]:
                    if hasattr(message, "content") and message.content:
                        content = str(message.content)
                        # Skip generic system messages and human messages
                        if (hasattr(message, "__class__") and "Human" not in message.__class__.__name__ and
                            not any(phrase in content.lower() for phrase in
                                   ["iteration limit", "time limit", "agent stopped"])):
                            response_parts.append(content)

            # If we have response parts, join them
            if response_parts:
                return "\n\n".join(response_parts)

            # Fallback to full result conversion
            result_str = str(result)

            # If result is a dict, try to extract useful parts
            if isinstance(result, dict):
                useful_parts = []
                for key in ['output', 'response', 'result', 'answer']:
                    if key in result and result[key]:
                        useful_parts.append(f"{key.title()}: {result[key]}")

                if useful_parts:
                    return "\n".join(useful_parts)

            return result_str

        except Exception as e:
            return f"Error extracting response: {e}"

    def run_single_evaluation(self, query: str) -> Dict[str, Any]:
        """Run evaluation for a single query - no manual validation."""
        if not self.agent:
            raise RuntimeError("Agent not initialized. Call setup_agent() first.")

        logger.info(f"🔍 Evaluating query: {query}")

        start_time = time.time()

        try:
            # Build starting state and run query
            state = FlightSearchGraph.build_starting_state(query=query)
            result = self.agent.invoke(state)

            # Extract response content
            response = self._extract_response_content(result)

            # Create evaluation result - no manual scoring
            evaluation_result = {
                "query": query,
                "response": response,
                "execution_time": time.time() - start_time,
                "success": True,
            }

            logger.info(f"✅ Query completed in {evaluation_result['execution_time']:.2f}s")
            return evaluation_result

        except Exception as e:
            logger.exception(f"❌ Query failed: {e}")
            return {
                "query": query,
                "response": f"Error: {str(e)}",
                "execution_time": time.time() - start_time,
                "success": False,
                "error": str(e),
            }

    def _create_reference_text(self, query: str) -> str:
        """Create reference text for evaluation based on query."""
        # Get the actual reference answer for this query
        reference_answer = get_reference_answer(query)

        if reference_answer.startswith("No reference answer available"):
            raise ValueError(
                f"No reference answer available for query: '{query}'. "
                f"Please add this query to QUERY_REFERENCE_ANSWERS in the queries cell"
            )

        return reference_answer

    def run_phoenix_evaluations(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Run Phoenix evaluations on the results."""
        if not ARIZE_AVAILABLE or not self.evaluators:
            logger.warning("⚠️ Phoenix evaluators not available - skipping evaluations")
            return results_df

        logger.info(f"🧠 Running Phoenix evaluations on {len(results_df)} responses...")
        logger.info("📋 Evaluation criteria:")
        logger.info("   🔍 Relevance: Does the response address the flight search query?")
        logger.info("   🎯 QA Correctness: Is the flight information accurate and helpful?")
        logger.info("   🚨 Hallucination: Does the response contain fabricated information?")
        logger.info("   ☠️ Toxicity: Is the response harmful or inappropriate?")

        try:
            # Prepare evaluation data
            evaluation_data = []
            for _, row in results_df.iterrows():
                query = row["query"]
                response = row["response"]

                # Create reference text based on query type (with validation)
                reference = self._create_reference_text(str(query))

                evaluation_data.append(
                    {
                        "input": query,
                        "output": response,
                        "reference": reference,
                        "query": query,  # For hallucination evaluation
                        "response": response,  # For hallucination evaluation
                        "text": response,  # For toxicity evaluation
                    }
                )

            eval_df = pd.DataFrame(evaluation_data)

            # Run individual Phoenix evaluations
            self._run_individual_phoenix_evaluations(eval_df, results_df)

            logger.info("✅ Phoenix evaluations completed")

        except Exception as e:
            logger.exception(f"❌ Error running Phoenix evaluations: {e}")
            # Add error indicators
            for eval_type in ["relevance", "qa_correctness", "hallucination", "toxicity"]:
                results_df[eval_type] = "error"
                results_df[f"{eval_type}_explanation"] = f"Error: {e}"

        return results_df

    def _run_individual_phoenix_evaluations(
        self, eval_df: pd.DataFrame, results_df: pd.DataFrame
    ) -> None:
        """Run individual Phoenix evaluations."""
        for eval_name, evaluator in self.evaluators.items():
            try:
                logger.info(f"   📊 Running {eval_name} evaluation...")

                # Prepare data based on evaluator requirements
                if eval_name == "relevance":
                    data = eval_df[["input", "reference"]].copy()
                    eval_results = llm_classify(
                        data=data,
                        model=self.evaluator_llm,
                        template=RAG_RELEVANCY_PROMPT_TEMPLATE,
                        rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
                        provide_explanation=True,
                    )
                elif eval_name == "qa_correctness":
                    data = eval_df[["input", "output", "reference"]].copy()
                    eval_results = llm_classify(
                        data=data,
                        model=self.evaluator_llm,
                        template=LENIENT_QA_PROMPT_TEMPLATE,
                        rails=LENIENT_QA_RAILS,
                        provide_explanation=True,
                    )
                elif eval_name == "hallucination":
                    data = eval_df[["input", "reference", "output"]].copy()
                    eval_results = llm_classify(
                        data=data,
                        model=self.evaluator_llm,
                        template=LENIENT_HALLUCINATION_PROMPT_TEMPLATE,
                        rails=LENIENT_HALLUCINATION_RAILS,
                        provide_explanation=True,
                    )
                elif eval_name == "toxicity":
                    data = eval_df[["input"]].copy()
                    eval_results = llm_classify(
                        data=data,
                        model=self.evaluator_llm,
                        template=TOXICITY_PROMPT_TEMPLATE,
                        rails=list(TOXICITY_PROMPT_RAILS_MAP.values()),
                        provide_explanation=True,
                    )
                else:
                    logger.warning(f"⚠️ Unknown evaluator: {eval_name}")
                    continue

                # Process results
                self._process_evaluation_results(eval_results, eval_name, results_df)

            except Exception as e:
                logger.warning(f"⚠️ {eval_name} evaluation failed: {e}")
                results_df[eval_name] = "error"
                results_df[f"{eval_name}_explanation"] = f"Error: {e}"

    def _process_evaluation_results(
        self, eval_results: Any, eval_name: str, results_df: pd.DataFrame
    ) -> None:
        """Process evaluation results and add to results DataFrame."""
        try:
            if eval_results is None:
                logger.warning(f"⚠️ {eval_name} evaluation returned None")
                results_df[eval_name] = "unknown"
                results_df[f"{eval_name}_explanation"] = "Evaluation returned None"
                return

            # Handle DataFrame results
            if hasattr(eval_results, "columns"):
                if "label" in eval_results.columns:
                    results_df[eval_name] = eval_results["label"].tolist()
                elif "classification" in eval_results.columns:
                    results_df[eval_name] = eval_results["classification"].tolist()
                else:
                    results_df[eval_name] = "unknown"

                if "explanation" in eval_results.columns:
                    results_df[f"{eval_name}_explanation"] = eval_results["explanation"].tolist()
                elif "reason" in eval_results.columns:
                    results_df[f"{eval_name}_explanation"] = eval_results["reason"].tolist()
                else:
                    results_df[f"{eval_name}_explanation"] = "No explanation provided"

                logger.info(f"   ✅ {eval_name} evaluation completed")

            # Handle list results
            elif isinstance(eval_results, list) and len(eval_results) > 0:
                if isinstance(eval_results[0], dict):
                    results_df[eval_name] = [item.get("label", "unknown") for item in eval_results]
                    results_df[f"{eval_name}_explanation"] = [
                        item.get("explanation", "No explanation") for item in eval_results
                    ]
                else:
                    results_df[eval_name] = eval_results
                    results_df[f"{eval_name}_explanation"] = "List evaluation result"

                logger.info(f"   ✅ {eval_name} evaluation completed (list format)")

            else:
                logger.warning(f"⚠️ {eval_name} evaluation returned unexpected format")
                results_df[eval_name] = "unknown"
                results_df[f"{eval_name}_explanation"] = f"Unexpected format: {type(eval_results)}"

        except Exception as e:
            logger.warning(f"⚠️ Error processing {eval_name} results: {e}")
            results_df[eval_name] = "error"
            results_df[f"{eval_name}_explanation"] = f"Processing error: {e}"

    def run_evaluation(self, queries: List[str]) -> pd.DataFrame:
        """Run complete evaluation pipeline using only Phoenix evaluators."""
        # Clear existing bookings for a clean test run
        clear_bookings_and_reviews()

        if not self.setup_agent():
            raise RuntimeError("Failed to setup agent")

        logger.info(f"🚀 Starting evaluation with {len(queries)} queries")

        # Log available features
        logger.info("📋 Evaluation Configuration:")
        logger.info(f"   🤖 Agent: Flight Search Agent (LangGraph)")
        logger.info(f"   🔧 Phoenix Available: {'✅' if ARIZE_AVAILABLE else '❌'}")
        logger.info(f"   📊 Arize Datasets: {'✅' if ARIZE_AVAILABLE and (self.dataset_manager.client is not None) else '❌'}")
        if self.evaluators:
            logger.info(f"   🧠 Phoenix Evaluators: {list(self.evaluators.keys())}")
        else:
            logger.info("   🧠 Phoenix Evaluators: ❌ (basic metrics only)")

        # Run queries (no manual validation)
        results = []
        for i, query in enumerate(queries, 1):
            logger.info(f"\n📋 Query {i}/{len(queries)}")
            result = self.run_single_evaluation(query)
            results.append(result)

        # Create results DataFrame
        results_df = pd.DataFrame(results)

        # Run Phoenix evaluations only
        results_df = self.run_phoenix_evaluations(results_df)

        # Log summary
        self._log_evaluation_summary(results_df)

        # Create Arize dataset
        dataset_id = self.dataset_manager.create_dataset(results_df)
        if dataset_id:
            logger.info(f"📊 Arize dataset created: {dataset_id}")
        else:
            logger.warning("⚠️ Dataset creation failed")

        return results_df

    def _log_evaluation_summary(self, results_df: pd.DataFrame) -> None:
        """Log evaluation summary using Phoenix results only."""
        logger.info("\n📊 Phoenix Evaluation Summary:")
        logger.info(f"  Total queries: {len(results_df)}")
        logger.info(f"  Successful executions: {results_df['success'].sum()}")
        logger.info(f"  Failed executions: {(~results_df['success']).sum()}")
        logger.info(f"  Average execution time: {results_df['execution_time'].mean():.2f}s")

        # Phoenix evaluation results
        if ARIZE_AVAILABLE and self.evaluators:
            logger.info("\n🧠 Phoenix Evaluation Results:")
            for eval_type in ["relevance", "qa_correctness", "hallucination", "toxicity"]:
                if eval_type in results_df.columns:
                    counts = results_df[eval_type].value_counts()
                    logger.info(f"   {eval_type}: {dict(counts)}")

        # Quick scores summary
        if len(results_df) > 0:
            logger.info("\n📊 Quick Scores Summary:")
            for i in range(len(results_df)):
                row = results_df.iloc[i]
                scores = []
                for eval_type in ["relevance", "qa_correctness", "hallucination", "toxicity"]:
                    if eval_type in row:
                        result = row[eval_type]
                        emoji = "✅" if result in ["relevant", "correct", "factual", "non-toxic"] else "❌"
                        scores.append(f"{emoji} {eval_type}: {result}")

                logger.info(f"   Query {i+1}: {' | '.join(scores)}")

        # Sample results with FULL detailed explanations for debugging
        if len(results_df) > 0:
            logger.info("\n📝 DETAILED EVALUATION RESULTS (FULL EXPLANATIONS):")
            logger.info("="*80)
            for i in range(min(len(results_df), len(results_df))):  # Show all results
                row = results_df.iloc[i]
                logger.info(f"\n🔍 QUERY {i+1}: {row['query']}")
                logger.info("-"*60)

                for eval_type in ["relevance", "qa_correctness", "hallucination", "toxicity"]:
                    if eval_type in row:
                        result = row[eval_type]
                        # Show FULL explanation instead of processed/truncated version
                        full_explanation = str(row.get(f"{eval_type}_explanation", "No explanation provided"))
                        logger.info(f"\n📊 {eval_type.upper()}: {result}")
                        logger.info(f"💭 FULL REASONING:")
                        logger.info(f"{full_explanation}")
                        logger.info("-"*40)
                logger.info("="*80)

    def cleanup(self) -> None:
        """Clean up all resources."""
        self.phoenix_manager.cleanup()

## Evaluator Methods - Part 2

Additional methods for the ArizeFlightSearchEvaluator class.


## Utility Functions

Utility functions for running evaluations and demos.


In [35]:
def get_default_queries() -> List[str]:
    """Get default test queries for evaluation."""
    # Use the same queries from data/queries.py to ensure reference answers match
    return FLIGHT_SEARCH_QUERIES

def run_phoenix_demo() -> pd.DataFrame:
    """Run a simple Phoenix evaluation demo."""
    logger.info("🔧 Running Phoenix evaluation demo...")

    demo_queries = [
        "Find flights from JFK to LAX",
        "What do passengers say about SpiceJet's service quality?",
    ]

    evaluator = ArizeFlightSearchEvaluator()
    try:
        results = evaluator.run_evaluation(demo_queries)
        logger.info("🎉 Phoenix evaluation demo complete!")
        logger.info("💡 Visit Phoenix UI to see detailed traces and evaluations")
        return results
    finally:
        evaluator.cleanup()

def run_full_evaluation() -> pd.DataFrame:
    """Main evaluation function using only Phoenix evaluators."""
    evaluator = ArizeFlightSearchEvaluator()
    try:
        results = evaluator.run_evaluation(get_default_queries())
        logger.info("\n✅ Phoenix evaluation complete!")
        return results
    finally:
        evaluator.cleanup()


## Run Evaluation

Execute the flight search agent evaluation with Phoenix AI.


In [36]:
# Run the evaluation
if __name__ == "__main__":
    # Choose evaluation mode:

    # Option 1: Run demo with 2 queries (quick test)
    # results = run_phoenix_demo()

    # Option 2: Run full evaluation with all queries
    results = run_full_evaluation()

    # Display results
    if results is not None and len(results) > 0:
        print("\n📊 EVALUATION RESULTS:")
        print("=" * 50)
        for i, row in results.iterrows():
            print(f"\n🔍 Query {i+1}: {row['query']}")
            print(f"⏱️ Execution time: {row['execution_time']:.2f}s")
            print(f"✅ Success: {row['success']}")

            # Show evaluation scores if available
            eval_types = ["relevance", "qa_correctness", "hallucination", "toxicity"]
            for eval_type in eval_types:
                if eval_type in row:
                    score = row[eval_type]
                    emoji = "✅" if score in ["relevant", "correct", "factual", "non-toxic"] else "❌"
                    print(f"   {emoji} {eval_type.title()}: {score}")

        print("\n🎉 Evaluation completed! Check Phoenix UI for detailed traces.")
    else:
        print("❌ No evaluation results generated.")

INFO:__main__:✅ Arize datasets client initialized successfully
INFO:__main__:✅ Phoenix evaluators initialized successfully
INFO:__main__:   🤖 Using evaluator model: gpt-4o
INFO:__main__:   📊 Available evaluators: ['relevance', 'qa_correctness', 'hallucination', 'toxicity']
INFO:__main__:🔧 Setting up Phoenix observability...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /root/.phoenix
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume transactional DDL.
INFO:alembic.runtime.migration:Running upgrade  -> cf03bd6bae1d, init
INFO:alembic.runtime.migration:Running upgrade cf03bd6bae1d -> 10460e46d750, datasets
INFO:alembic.runtime.migration:Running upgrade 10460e46d750 -> 3be8647b87d8, add token columns to spans table
INFO:alembic.runtime.migration:Running upgrade 3be8647b87d8 -> cd164e83824f, users and tokens
INFO:alembic.runtime.migration:Running upgrade cd164e83824f -> 4ded9e43755f, create project_session table
INFO:alembic.runt

🌍 To view the Phoenix app in your browser, visit https://ducjn7hsnyw1-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


INFO:__main__:🌐 Phoenix UI: https://ducjn7hsnyw1-496ff2e9c6d22116-6006-colab.googleusercontent.com/
INFO:__main__:✅ Phoenix setup completed successfully
INFO:__main__:✅ LangChain instrumentation enabled
INFO:__main__:✅ OpenAI instrumentation enabled


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: flight-search-agent-evaluation
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



INFO:__main__:✅ Successfully connected to Couchbase
INFO:__main__:🗑️ Clearing data from travel-sample.agentc_bookings.user_bookings_20251023...
INFO:__main__:🗑️ Clearing data from travel-sample.agentc_bookings.user_bookings_20251022...
INFO:__main__:✅ Completed clearing scope: travel-sample.agentc_bookings
INFO:__main__:✅ Cleared existing flight bookings for fresh test run: travel-sample.agentc_bookings
INFO:__main__:📊 Airline reviews collection: 2210 existing, 2210 expected
INFO:__main__:✅ Collection already has correct document count (2210), skipping clear
INFO:__main__:🔧 Setting up flight search agent...
INFO:__main__:✅ Environment variables configured
INFO:__main__:✅ Successfully connected to Couchbase
INFO:__main__:✅ Collection setup complete: agentc_data.airline_reviews
INFO:__main__:Loaded vector search index definition from agentcatalog_index.json
INFO:__main__:ℹ️ Vector search index 'airline_reviews_index' already exists
INFO:__main__:🔧 Setting up Priority 1 AI services for la

...

llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

INFO:__main__:   ✅ hallucination evaluation completed
INFO:__main__:   📊 Running toxicity evaluation...


llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

INFO:__main__:   ✅ toxicity evaluation completed
INFO:__main__:✅ Phoenix evaluations completed
INFO:__main__:
📊 Phoenix Evaluation Summary:
INFO:__main__:  Total queries: 5
INFO:__main__:  Successful executions: 5
INFO:__main__:  Failed executions: 0
INFO:__main__:  Average execution time: 3.64s
INFO:__main__:
🧠 Phoenix Evaluation Results:
INFO:__main__:   relevance: {'relevant': np.int64(5)}
INFO:__main__:   qa_correctness: {'correct': np.int64(5)}
INFO:__main__:   hallucination: {'factual': np.int64(5)}
INFO:__main__:   toxicity: {'non-toxic': np.int64(5)}
INFO:__main__:
📊 Quick Scores Summary:
INFO:__main__:   Query 1: ✅ relevance: relevant | ✅ qa_correctness: correct | ✅ hallucination: factual | ✅ toxicity: non-toxic
INFO:__main__:   Query 2: ✅ relevance: relevant | ✅ qa_correctness: correct | ✅ hallucination: factual | ✅ toxicity: non-toxic
INFO:__main__:   Query 3: ✅ relevance: relevant | ✅ qa_correctness: correct | ✅ hallucination: factual | ✅ toxicity: non-toxic
INFO:__main__: 


📊 EVALUATION RESULTS:

🔍 Query 1: Find flights from JFK to LAX
⏱️ Execution time: 2.86s
✅ Success: True
   ✅ Relevance: relevant
   ✅ Qa_Correctness: correct
   ✅ Hallucination: factual
   ✅ Toxicity: non-toxic

🔍 Query 2: Book a flight from LAX to JFK for tomorrow, 2 passengers, business class
⏱️ Execution time: 4.11s
✅ Success: True
   ✅ Relevance: relevant
   ✅ Qa_Correctness: correct
   ✅ Hallucination: factual
   ✅ Toxicity: non-toxic

🔍 Query 3: Book an economy flight from JFK to MIA for next week, 1 passenger
⏱️ Execution time: 4.05s
✅ Success: True
   ✅ Relevance: relevant
   ✅ Qa_Correctness: correct
   ✅ Hallucination: factual
   ✅ Toxicity: non-toxic

🔍 Query 4: Show me my current flight bookings
⏱️ Execution time: 3.11s
✅ Success: True
   ✅ Relevance: relevant
   ✅ Qa_Correctness: correct
   ✅ Hallucination: factual
   ✅ Toxicity: non-toxic

🔍 Query 5: What do passengers say about SpiceJet's service quality?
⏱️ Execution time: 4.07s
✅ Success: True
   ✅ Relevance: relevant

## Summary

This self-contained notebook demonstrates a complete flight search agent implementation using LangGraph framework with ReAct agents, Couchbase vector store for airline reviews, and Priority 1 Capella AI services. The agent handles flight search, booking, retrieval, and airline review queries with exact functionality from the working Python files. All components are inline and the notebook runs as a complete workflow when executed cell by cell. Set up environment variables (CB_*, CAPELLA_API_*) and dependencies, then execute all cells sequentially.
